In [1]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import torch.nn.functional as F
import os
import random
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
from sklearn import linear_model
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import RandomUnderSampler 
from imblearn.combine import SMOTEENN
from imblearn.combine import SMOTETomek
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
from sklearn.covariance import EllipticEnvelope
from sklearn.neighbors import LocalOutlierFactor
from sklearn import preprocessing


c:\Users\Windows\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
if __name__ == '__main__':
    root = os.path.dirname(os.getcwd())
    print("root direction", root)
    dataset = os.listdir(root + '/muldataset')
    del dataset[-1]
    invalid_num = 0
    for sample in dataset:
        sample_path = root + '/muldataset' + '/' + sample #样本文件夹
        # print(sample_path)
        sample_file = os.listdir(sample_path) #样本文件夹内文件列表
        # print(sample_file)
        for detail in sample_file:
            sample_path = root + '/muldataset' + '/' + sample
            if 'face_landmarks' in detail:
                df = pd.read_csv(sample_path + '/' + detail, usecols=[1,2,3])
                face_x = df['x'].values.tolist()
                num = np.array(face_x).shape[0] / 468
                if num <= 15:
                    invalid_num += 1
                    print(sample_path)
    print(invalid_num)

root direction f:\Project\AMH


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [32]:

class EmbraceNet(nn.Module):
  
  def __init__(self, device, input_size_list, embracement_size=256, bypass_docking=False):
    """
    Initialize an EmbraceNet module.
    Args:
      device: A "torch.device()" object to allocate internal parameters of the EmbraceNet module.
      input_size_list: A list of input sizes.
      embracement_size: The length of the output of the embracement layer ("c" in the paper).
      bypass_docking: Bypass docking step, i.e., connect the input data directly to the embracement layer. If True, input_data must have a shape of [batch_size, embracement_size].
    """
    super(EmbraceNet, self).__init__()

    self.device = device
    self.input_size_list = input_size_list
    self.embracement_size = embracement_size
    self.bypass_docking = bypass_docking
    # self.drop = nn.Dropout(p=0.5)
  
    if (not bypass_docking):
      for i, input_size in enumerate(input_size_list):
        # setattr(self, 'project_%d' % (i), nn.Linear(input_size, embracement_size // 2))
        setattr(self, 'docking_%d' % (i), nn.Linear(input_size, embracement_size))
 
  def forward(self, input_list, availabilities=None, selection_probabilities=None):
    """
    Forward input data to the EmbraceNet module.
    Args:
      input_list: A list of input data. Each input data should have a size as in input_size_list.
      availabilities: A 2-D tensor of shape [batch_size, num_modalities], which represents the availability of data for each modality. If None, it assumes that data of all modalities are available.
      selection_probabilities: A 2-D tensor of shape [batch_size, num_modalities], which represents probabilities that output of each docking layer will be selected ("p" in the paper). If None, the same probability of being selected will be used for each docking layer.
    Returns:
      A 2-D tensor of shape [batch_size, embracement_size] that is the embraced output.
    """

    # check input data
    assert len(input_list) == len(self.input_size_list)
    num_modalities = len(input_list)
    batch_size = input_list[0].shape[0]
    
    
    # docking layer
    docking_output_list = []
    distance = []
    if (self.bypass_docking):
      docking_output_list = input_list
    else:
      for i, input_data in enumerate(input_list):
        # x = getattr(self, 'project_%d' % (i))(input_data)
        x = getattr(self, 'docking_%d' % (i))(input_data)
        # x = self.drop(x)
        # dropped = self.drop(x)
      
        x = nn.functional.relu(x)
        docking_output_list.append(x)
     

    # check availabilities
    if (availabilities is None):
      availabilities = torch.ones(batch_size, len(input_list), dtype=torch.float, device=self.device)
    else:
      availabilities = availabilities.float()
    
    # adjust selection probabilities
    if (selection_probabilities is None):
      selection_probabilities = torch.ones(batch_size, len(input_list), dtype=torch.float, device=self.device)
    selection_probabilities = torch.mul(selection_probabilities, availabilities)
    
    probability_sum = torch.sum(selection_probabilities, dim=-1, keepdim=True)
    selection_probabilities = torch.div(selection_probabilities, probability_sum)
    # stack docking outputs
    docking_output_stack = torch.stack(docking_output_list, dim=-1)  # [batch_size, embracement_size, num_modalities]

    # print("Batch size:",batch_size)
    # embrace
    modality_indices = torch.multinomial(selection_probabilities, num_samples=self.embracement_size, replacement=True)  # [batch_size, embracement_size]
    modality_toggles = nn.functional.one_hot(modality_indices, num_classes=num_modalities).float()  # [batch_size, embracement_size, num_modalities]

    embracement_output_stack = torch.mul(docking_output_stack, modality_toggles)
    embracement_output = torch.sum(embracement_output_stack, dim=-1)  # [batch_size, embracement_size]
    return embracement_output


In [7]:
class EmbraceTextAudioVideo(nn.Module):
    def __init__(self, device, input_size, embrace_size):
        super(EmbraceTextAudioVideo, self).__init__()

        self.device = device
        # embracenet
        self.embracenet = EmbraceNet(device=self.device, input_size_list=input_size, embracement_size=embrace_size)

        # post embracement layers
        self.post = nn.Linear(in_features=embrace_size, out_features=2)

        # self.acti = nn.Tanhshrink()
        self.acti = nn.Hardtanh()
        # self.acti = nn.ELU()
        # self.acti = nn.Softsign()
        # self.acti = nn.LogSigmoid()
        # self.acti = nn.Tanh()

    def forward(self, roberta, pert, macbert, wav, iris, physical_index, face):
        embraced_output = self.embracenet(input_list=[roberta, pert, macbert, wav, iris, physical_index, face])
        # embraced_output = self.embracenet(input_list=[roberta, pert, macbert, wav, iris, physical_index])
        x = self.post(embraced_output)
        # print(x)
        # x = torch.sigmoid(x)
        # x = torch.relu(x)
        # x = torch.rrelu(x)
        # x = torch.selu(x)
        # x = torch.celu(x)
        x = self.acti(x)
        return x



In [40]:
root = os.path.dirname(os.getcwd())
print("root direction", root)

dataset = os.listdir(root + '/muldataset')
del dataset[-1]
# print(dataset)

for sample in dataset:
    sample_path = root + '/muldataset' + '/' + sample
    
    sample_file = os.listdir(sample_path)
    # print(sample_file)
    if 'text_feature.csv' in sample_file:
        continue
    else:
        print(sample_path)

 

root direction f:\Project\AMH


In [158]:
root = os.path.dirname(os.getcwd())
print("root direction", root)
dataset = os.listdir(root + '/muldataset')
del dataset[-2:]


label_path = root + '/muldataset/label.csv'
df = pd.read_csv(label_path)
folder_name = df['folder'].values.tolist()
print(len(folder_name))

diff = list(set(folder_name) ^ set(dataset))
print(diff)

root direction f:\Project\AMH
1101
[]


In [35]:
root = os.path.dirname(os.getcwd())
print("root direction", root)
dataset = os.listdir(root + '/muldataset')
del dataset[-1]
del dataset[-1]

label_path = root + '/muldataset/copy_labels.csv'
df = pd.read_csv(label_path)
# print(dataset)
folder_name = df['folder'].values.tolist()
folder_index = []
drop_index = []
print(folder_name.index('00004-0104'))

for sample in dataset:
    if sample in folder_name:
        folder_index.append(folder_name.index(sample))
    else:
        print(sample)
print(np.array(folder_index).shape)

diff = list(set(folder_name) ^ set(dataset))
print(np.array(diff).shape)
for i in diff:
    drop_index.append(folder_name.index(i))
print(drop_index)
drop_index = sorted(drop_index, reverse=True)
print(drop_index)

for i in drop_index:
    df.drop(df.index[i],  inplace=True)
df.to_csv(root + '/muldataset/label.csv', index=False, encoding="gbk")


root direction f:\Project\AMH
2
(1107,)
(412,)
[39, 341, 284, 449, 569, 885, 613, 884, 433, 72, 533, 776, 694, 983, 799, 619, 507, 16, 489, 516, 329, 521, 204, 261, 524, 727, 243, 708, 335, 755, 876, 633, 380, 109, 282, 937, 740, 404, 567, 1017, 558, 273, 259, 364, 378, 563, 467, 882, 1498, 55, 1027, 984, 574, 553, 875, 62, 809, 413, 306, 989, 488, 421, 255, 327, 1019, 245, 208, 899, 692, 119, 821, 1049, 167, 154, 485, 48, 702, 464, 562, 466, 652, 816, 958, 131, 505, 857, 416, 239, 700, 838, 242, 517, 916, 498, 1038, 18, 1029, 631, 285, 846, 1008, 397, 491, 512, 496, 854, 585, 434, 249, 479, 75, 580, 933, 939, 711, 949, 748, 229, 907, 665, 200, 683, 143, 961, 406, 389, 749, 268, 781, 753, 554, 906, 277, 659, 124, 688, 990, 686, 713, 205, 657, 81, 671, 787, 355, 195, 0, 890, 845, 766, 1, 149, 7, 593, 330, 764, 663, 733, 1298, 116, 310, 832, 1366, 111, 514, 556, 1304, 132, 44, 813, 609, 185, 675, 473, 522, 487, 655, 896, 258, 334, 78, 565, 515, 664, 1006, 868, 544, 396, 630, 691, 236, 45

In [355]:
root = os.path.dirname(os.getcwd())
print("root direction", root)

label_path = root + '/muldataset/label.csv'
print(label_path)

df = pd.read_csv(label_path)
# print(df)
df_folder = df['folder'].values.tolist()
df_dep = df['depression'].values.tolist()
df_inter = df['interpersonal tension'].values.tolist()
df_anx = df['anxiety'].values.tolist()
df_obs = df['obsessive-compulsive'].values.tolist()
df_par = df['paranoia'].values.tolist()
df_hos = df['hostility'].values.tolist()
df_aca = df['academic stress'].values.tolist()
df_mal = df['maladjustment'].values.tolist()
df_emo = df['emotional imbalance'].values.tolist()
df_psy = df['psychological imbalance'].values.tolist()
df_label = df['overall condition'].values.tolist()
df_sui = df['suicidal tendency'].values.tolist()
# print(df_folder)
print(df_folder[409])
# labels = df_label[:410]
# labels = df_label[410:]
labels = df_label
print(np.array(labels).shape)

root direction f:\Project\AMH
f:\Project\AMH/muldataset/label.csv
00004-0104
(1101,)


In [263]:
delete_folder_index = []

for i in wrong_num_face:
    if i in wrong_num_face_vari and i in wrong_num_iris and i in wrong_num_mfcc and i in wrong_num_wav and i in wrong_num_phy and i in wrong_num_pert and i in wrong_num_rob:
        delete_folder_index.append(i)

delete_folder_index = np.array(delete_folder_index).reshape(1,-1)[0]
print(np.array(delete_folder_index))

[   3    9   19   20   28   29   31   41   42   58   70   90   95  106
  111  112  114  117  119  135  136  145  146  148  166  168  172  174
  175  178  191  193  194  197  199  204  205  207  210  212  214  215
  220  222  224  225  226  228  233  237  242  243  246  256  260  263
  266  269  276  278  279  288  289  294  298  299  306  309  310  311
  313  314  315  326  329  336  339  342  353  354  364  370  372  373
  374  385  386  387  388  390  391  396  403  419  421  422  423  430
  436  438  443  444  450  452  463  476  483  485  487  494  511  513
  514  518  522  523  526  528  534  537  539  540  554  558  560  568
  573  579  584  590  607  609  612  621  622  624  628  631  632  634
  639  641  643  648  659  661  664  665  673  677  678  686  690  691
  695  696  697  698  709  710  713  714  715  730  735  742  744  747
  749  753  755  758  759  762  765  771  774  776  783  787  790  796
  805  808  810  811  815  816  818  819  825  827  829  832  833  834
  840 

In [360]:
# 写入csv
name = ['delete_index']
landmarks = pd.DataFrame(columns=name, data=delete_folder_index)

landmarks.to_csv(root + '/muldataset/delete_index.csv', encoding='gbk')

In [357]:
root = os.path.dirname(os.getcwd())

df = pd.read_csv(root + '/muldataset/wrong_num.csv', usecols=[1])
df_wrg = df['delete_index'].values.tolist()
delete_folder_index = df_wrg


In [358]:
random.seed(0)
random.shuffle(delete_folder_index)

delete_folder_index = delete_folder_index
del delete_folder_index[132:-1] 
print(np.array(delete_folder_index).shape)
print(delete_folder_index)

(133,)
[294, 765, 269, 893, 212, 225, 276, 956, 1008, 353, 309, 514, 981, 237, 749, 1083, 1010, 9, 215, 278, 957, 528, 744, 833, 584, 1081, 214, 372, 1028, 1024, 643, 135, 339, 423, 834, 386, 1017, 959, 256, 631, 919, 511, 710, 243, 609, 106, 222, 1079, 70, 805, 191, 226, 664, 816, 314, 927, 299, 31, 1058, 742, 579, 194, 421, 977, 624, 224, 659, 422, 852, 898, 811, 483, 494, 311, 403, 849, 172, 696, 388, 840, 774, 537, 612, 28, 526, 714, 90, 242, 534, 476, 430, 673, 438, 1055, 863, 964, 387, 762, 310, 698, 715, 29, 818, 390, 969, 735, 95, 41, 463, 260, 558, 1043, 444, 686, 175, 1095, 111, 199, 419, 19, 755, 1020, 487, 220, 910, 336, 827, 178, 819, 145, 1060, 1080, 845]


In [359]:
# labels = labels.tolist()
for i in range(len(labels), 0, -1):
    if i in delete_folder_index:
        del labels[i]
print(np.array(labels).shape)

(968,)


In [361]:
from dtaidistance import dtw
def attention_add(a, b, w_ab):
    len_a = len(a)
    len_b = len(b)
    
    if len_a > len_b:
        distance, paths = dtw.warping_paths(a, b)
        best_path = dtw.best_path(paths)
        # print(best_path)
        for i in best_path:
            a[i[0]] += w_ab * b[i[1]]

        # for i in range(0, len(a) - len(b)):
        #     temp_dis = dtw.distance(a[i:i+len(np.array(b))], b)
        #     if temp_dis < dis:
        #         dis = temp_dis
        #         startidx = i
        #         endidx = i + len(np.array(b))
        # output = a[:startidx].tolist() + (a[startidx:endidx] + w_ab * b).tolist() + a[endidx+1:]
    elif len_a < len_b:
        distance, paths = dtw.warping_paths(a, b)
        best_path = dtw.best_path(paths)
        # print(best_path)
        for i in best_path:
            a[i[0]] += w_ab * b[i[1]]

        # for i in range(0, len(b) - len(a)):
        #     temp_dis = dtw.distance(b[i:i+len(np.array(a))], a)
        #     if temp_dis < dis:
        #         dis = temp_dis
        #         startidx = i
        #         endidx = i + len(np.array(a))
        # output = (a + w_ab * b[startidx:endidx]).tolist()
    else:     
        a = (a + w_ab*b)
        # print(a)
    
        
    # print(dis)
    # print(startidx, endidx)
    return a

def softmax(x):
    x = x - np.max(x)
    exp_x = np.exp(x)
    softmax_x = exp_x / np.sum(exp_x)
    return softmax_x

def dtw_attention(v1, v2, v3, v4, v5, v6, v7, v8):
    v1 = np.array(v1)
    v2 = np.array(v2)
    v3 = np.array(v3)
    v4 = np.array(v4)
    v5 = np.array(v5)
    v6 = np.array(v6)
    v7 = np.array(v7)
    v8 = np.array(v8)

    w12 = dtw.distance_fast(v1, v2)
    w13 = dtw.distance_fast(v1, v3)
    w14 = dtw.distance_fast(v1, v4)
    w15 = dtw.distance_fast(v1, v5)
    w16 = dtw.distance_fast(v1, v6)
    w17 = dtw.distance_fast(v1, v7)
    w18 = dtw.distance_fast(v1, v8)

    w = [1/w12, 1/w13, 1/w14, 1/w15, 1/w16, 1/w17, 1/w18]
    w = softmax(w)
    # print(w)
    
    v1 = attention_add(v1, v2, w[0])
    print("2")
    v1 = attention_add(v1, v3, w[1])
    print("3")
    v1 = attention_add(v1, v4, w[2])
    print("4")
    v1 = attention_add(v1, v5, w[3])
    print("5")
    v1 = attention_add(v1, v6, w[4])
    print("6")
    v1 = attention_add(v1, v7, w[5])
    print("7")
    v1 = attention_add(v1, v8, w[6])
    print("8")
    # output = v1.tolist() + (w12*v2).tolist() + (w13*v3).tolist() + (w14*v4).tolist() + (w15*v5).tolist() + (w16*v6).tolist() + (w17*v7).tolist() + (w18*v8).tolist() + (w19*v9).tolist()
    # print(np.array(output).shape)  
    return v1

def get_face_vari(csv_path):
    df = pd.read_csv(csv_path, usecols=[1])
    df_face = df['face_vari'].values.tolist()
    
    return df_face

def get_face(csv_path):
    df = pd.read_csv(csv_path, usecols=[1])
    df_face = df['face'].values.tolist()
    
    return df_face

def get_iris(csv_path):
    df = pd.read_csv(csv_path, usecols=[1])
    df_iris = df['iris'].values.tolist()
    
    return df_iris

def get_physical_index(csv_path):
    df = pd.read_csv(csv_path, usecols=[1])
    df_index = df['physical_index'].values.tolist()

    return df_index

def get_voice_feature(csv_path):
    df = pd.read_csv(csv_path, usecols=[1])
    df_mfcc = df['mfcc'].values.tolist()
    
    return df_mfcc

def get_wav2vec(csv_path):
    df = pd.read_csv(csv_path, usecols=[1])
    df_wav2vec = df['wav2vec'].values.tolist()
    
    return df_wav2vec

def get_pert_feature(csv_path):
    df = pd.read_csv(csv_path, usecols=[1])
    df_pert = df['pert_att'].values.tolist()
    return df_pert

def get_roberta_feature(csv_path):
    df = pd.read_csv(csv_path, usecols=[1])
    df_roberta = df['robera_att'].values.tolist()
    return df_roberta

def get_text_feature(csv_path):
    df = pd.read_csv(csv_path, usecols=[1,2])
    df_pert = df['pert'].values.tolist()
    df_roberta = df['roberta'].values.tolist()
    return df_pert, df_roberta

def get_graph(csv_path):
    df = pd.read_csv(csv_path, usecols=[1])
    df_graph = df['graph'].values.tolist()
    return df_graph


if __name__ == '__main__':
    face = []
    face_vari = []
    iris = []
    physical_index = []
    mfcc_ft = []
    wav2vec = []
    pert = []
    roberta = []
    graph = []

    sample_index = 0
    root = os.path.dirname(os.getcwd())
    print("root direction", root)
    dataset = os.listdir(root + '/muldataset')
    del dataset[-3:]
    for sample in dataset:
        if sample_index in delete_folder_index:
            sample_index += 1
            continue
        sample_path = root + '/muldataset' + '/' + sample #样本文件夹
        # print(sample_path)
        sample_file = os.listdir(sample_path) #样本文件夹内文件列表
        # print(sample_file)
        # if 'roberta_att.csv' in sample_file:
        #     continue
        for detail in sample_file:
                sample_path = root + '/muldataset' + '/' + sample
                if 'fft_face.csv' in detail:
                    # 打开csv文件并读取人脸信息
                    print(sample_path + '/' + detail)
                    temp_face = get_face(sample_path + '/' + detail)
                    face.append(temp_face)
                    print(np.array(face).shape)

                if 'fft_face_vari.csv' in detail:
                    # 打开csv文件并读取人脸信息
                    print(sample_path + '/' + detail)
                    temp_face_vari = get_face_vari(sample_path + '/' + detail)
                    face_vari.append(temp_face_vari)
                    print(np.array(face_vari).shape)

                if 'fft_iris' in detail:
                    print(sample_path + '/' + detail)
                    temp_iris = get_iris(sample_path + '/' + detail)
                    iris.append(temp_iris)
                    print(np.array(iris).shape)

                if 'fft_physical_index' in detail:
                    print(sample_path + '/' + detail)
                    temp_index = get_physical_index(sample_path + '/' + detail)
                    physical_index.append(temp_index)
                    print(np.array(physical_index).shape)

                if 'fft_mfcc' in detail:
                    print(sample_path + '/' + detail)
                    temp_mfcc = get_voice_feature(sample_path + '/' + detail)
                    mfcc_ft.append(temp_mfcc)
                    print(np.array(mfcc_ft).shape)

                if 'fft_wav2vec' in detail:
                    print(sample_path + '/' + detail)
                    temp_wav2vec = get_wav2vec(sample_path + '/' + detail)
                    wav2vec.append(temp_wav2vec)
                    print(np.array(wav2vec).shape)

                if 'fft_text' in detail:
                    print(sample_path + '/' + detail)
                    temp_pert, temp_roberta = get_text_feature(sample_path + '/' + detail)
                    pert.append(temp_pert)
                    roberta.append(temp_roberta)
                    print(np.array(pert).shape, np.array(roberta).shape)
                
                if 'graph' in detail:
                    print(sample_path + '/' + detail)
                    temp_graph = get_graph(sample_path + '/' + detail)
                    graph.append(temp_graph)
                    print(np.array(graph).shape)
                    
        sample_index += 1
    print("Face set:{face}".format(face = np.array(face).shape))
    print("Face vari set:{face}".format(face = np.array(face_vari).shape))
    print("Iris set:{iris}".format(iris = np.array(iris).shape))
    print("Physical index set:{index}".format(index = np.array(physical_index).shape))
    print("MFCC set:{mfcc}".format(mfcc = np.array(mfcc_ft).shape))
    print("Pert set:{pert}, Roberta:{robert}".format(pert = np.array(pert).shape, \
            robert = np.array(roberta).shape))
    print("wav2vec set:{wav2vec}".format(wav2vec = np.array(wav2vec).shape))
    print("Graph set:{gra}".format(gra = np.array(graph).shape))
    print("Label set:{label}".format(label = np.array(labels).shape))    


root direction f:\Project\AMH
f:\Project\AMH/muldataset/00002-0101/fft_face.csv
(1, 1404)
f:\Project\AMH/muldataset/00002-0101/fft_face_vari.csv
(1, 1404)
f:\Project\AMH/muldataset/00002-0101/fft_iris.csv
(1, 2776)
f:\Project\AMH/muldataset/00002-0101/fft_mfcc.csv
(1, 12987)
f:\Project\AMH/muldataset/00002-0101/fft_physical_index.csv
(1, 4722)
f:\Project\AMH/muldataset/00002-0101/fft_text.csv
(1, 1024) (1, 1024)
f:\Project\AMH/muldataset/00002-0101/fft_wav2vec.csv
(1, 1024)
f:\Project\AMH/muldataset/00002-0101/graph.csv
(1, 64)
f:\Project\AMH/muldataset/00002-0102/fft_face.csv
(2, 1404)
f:\Project\AMH/muldataset/00002-0102/fft_face_vari.csv
(2, 1404)
f:\Project\AMH/muldataset/00002-0102/fft_iris.csv
(2, 2776)
f:\Project\AMH/muldataset/00002-0102/fft_mfcc.csv
(2, 12987)
f:\Project\AMH/muldataset/00002-0102/fft_physical_index.csv
(2, 4722)
f:\Project\AMH/muldataset/00002-0102/fft_text.csv
(2, 1024) (2, 1024)
f:\Project\AMH/muldataset/00002-0102/fft_wav2vec.csv
(2, 1024)
f:\Project\AMH/mu

In [362]:
from dtaidistance import dtw
def attention_add(a, b, w_ab):
    len_a = len(a)
    len_b = len(b)
    
    if len_a > len_b:
        distance, paths = dtw.warping_paths(a, b)
        best_path = dtw.best_path(paths)
        # print(best_path)
        for i in best_path:
            a[i[0]] += w_ab * b[i[1]]

        # for i in range(0, len(a) - len(b)):
        #     temp_dis = dtw.distance(a[i:i+len(np.array(b))], b)
        #     if temp_dis < dis:
        #         dis = temp_dis
        #         startidx = i
        #         endidx = i + len(np.array(b))
        # output = a[:startidx].tolist() + (a[startidx:endidx] + w_ab * b).tolist() + a[endidx+1:]
    elif len_a < len_b:
        distance, paths = dtw.warping_paths(a, b)
        best_path = dtw.best_path(paths)
        # print(best_path)
        for i in best_path:
            a[i[0]] += w_ab * b[i[1]]

        # for i in range(0, len(b) - len(a)):
        #     temp_dis = dtw.distance(b[i:i+len(np.array(a))], a)
        #     if temp_dis < dis:
        #         dis = temp_dis
        #         startidx = i
        #         endidx = i + len(np.array(a))
        # output = (a + w_ab * b[startidx:endidx]).tolist()
    else:     
        a = (a + w_ab*b)
        # print(a)
    
        
    # print(dis)
    # print(startidx, endidx)
    return a

def softmax(x):
    x = x - np.max(x)
    exp_x = np.exp(x)
    softmax_x = exp_x / np.sum(exp_x)
    return softmax_x

def dtw_attention(v1, v2, v3, v4, v5, v6, v7, v8):
    v1 = np.array(v1)
    v2 = np.array(v2)
    v3 = np.array(v3)
    v4 = np.array(v4)
    v5 = np.array(v5)
    v6 = np.array(v6)
    v7 = np.array(v7)
    v8 = np.array(v8)

    w12 = dtw.distance_fast(v1, v2)
    w13 = dtw.distance_fast(v1, v3)
    w14 = dtw.distance_fast(v1, v4)
    w15 = dtw.distance_fast(v1, v5)
    w16 = dtw.distance_fast(v1, v6)
    w17 = dtw.distance_fast(v1, v7)
    w18 = dtw.distance_fast(v1, v8)

    w = [1/w12, 1/w13, 1/w14, 1/w15, 1/w16, 1/w17, 1/w18]
    w = softmax(w)
    # print(w)
    
    v1 = attention_add(v1, v2, w[0])
    print("2")
    v1 = attention_add(v1, v3, w[1])
    print("3")
    v1 = attention_add(v1, v4, w[2])
    print("4")
    v1 = attention_add(v1, v5, w[3])
    print("5")
    v1 = attention_add(v1, v6, w[4])
    print("6")
    v1 = attention_add(v1, v7, w[5])
    print("7")
    v1 = attention_add(v1, v8, w[6])
    print("8")
    # output = v1.tolist() + (w12*v2).tolist() + (w13*v3).tolist() + (w14*v4).tolist() + (w15*v5).tolist() + (w16*v6).tolist() + (w17*v7).tolist() + (w18*v8).tolist() + (w19*v9).tolist()
    # print(np.array(output).shape)  
    return v1

def get_face_vari(csv_path):
    df = pd.read_csv(csv_path, usecols=[1])
    df_face = df['face_vari_att'].values.tolist()
    
    return df_face

def get_face(csv_path):
    df = pd.read_csv(csv_path, usecols=[1])
    df_face = df['face_att'].values.tolist()
    
    return df_face

def get_iris(csv_path):
    df = pd.read_csv(csv_path, usecols=[1])
    df_iris = df['iris_att'].values.tolist()
    
    return df_iris

def get_physical_index(csv_path):
    df = pd.read_csv(csv_path, usecols=[1])
    df_index = df['index_att'].values.tolist()

    return df_index

def get_voice_feature(csv_path):
    df = pd.read_csv(csv_path, usecols=[1])
    df_mfcc = df['mfcc_att'].values.tolist()
    
    return df_mfcc

def get_wav2vec(csv_path):
    df = pd.read_csv(csv_path, usecols=[1])
    df_wav2vec = df['wav2vec_att'].values.tolist()
    
    return df_wav2vec

def get_pert_feature(csv_path):
    df = pd.read_csv(csv_path, usecols=[1])
    df_pert = df['pert_att'].values.tolist()
    return df_pert

def get_roberta_feature(csv_path):
    df = pd.read_csv(csv_path, usecols=[1])
    df_roberta = df['robera_att'].values.tolist()
    return df_roberta

def get_graph(csv_path):
    df = pd.read_csv(csv_path, usecols=[1])
    df_graph = df['graph'].values.tolist()
    return df_graph


if __name__ == '__main__':
    face_att = []
    face_vari_att = []
    iris_att = []
    index_att = []
    mfcc_att = []
    wav2vec_att = []
    pert_att = []
    roberta_att = []
    # graph = []

    sample_index = 0
    root = os.path.dirname(os.getcwd())
    print("root direction", root)
    dataset = os.listdir(root + '/muldataset')
    del dataset[-3:]
    for sample in dataset:
        if sample_index in delete_folder_index:
            sample_index += 1
            continue
        # if '00002' in sample:
        #     continue
        # if '00004-0' in sample:
        #     continue
        # if '00004-19' not in sample:
        #     continue
        sample_path = root + '/muldataset' + '/' + sample #样本文件夹
        # print(sample_path)
        sample_file = os.listdir(sample_path) #样本文件夹内文件列表
        # print(sample_file)
        # if 'roberta_att.csv' in sample_file:
        #     continue
        for detail in sample_file:
                sample_path = root + '/muldataset' + '/' + sample
                if 'face_att.csv' in detail:
                    # 打开csv文件并读取人脸信息
                    print(sample_path + '/' + detail)
                    temp_face = get_face(sample_path + '/' + detail)
                    face_att.append(temp_face)
                    print(np.array(face_att).shape)

                if 'face_vari_att.csv' in detail:
                    # 打开csv文件并读取人脸信息
                    print(sample_path + '/' + detail)
                    temp_face_vari = get_face_vari(sample_path + '/' + detail)
                    face_vari_att.append(temp_face_vari)
                    print(np.array(face_vari_att).shape)

                if 'iris_att' in detail:
                    print(sample_path + '/' + detail)
                    temp_iris = get_iris(sample_path + '/' + detail)
                    iris_att.append(temp_iris)
                    print(np.array(iris_att).shape)

                if 'index_att' in detail:
                    print(sample_path + '/' + detail)
                    temp_index = get_physical_index(sample_path + '/' + detail)
                    index_att.append(temp_index)
                    print(np.array(index_att).shape)

                if 'mfcc_att' in detail:
                    print(sample_path + '/' + detail)
                    temp_mfcc = get_voice_feature(sample_path + '/' + detail)
                    mfcc_att.append(temp_mfcc)
                    print(np.array(mfcc_att).shape)

                if 'wav2vec_att' in detail:
                    print(sample_path + '/' + detail)
                    temp_wav2vec = get_wav2vec(sample_path + '/' + detail)
                    wav2vec_att.append(temp_wav2vec)
                    print(np.array(wav2vec_att).shape)

                if 'pert_att' in detail:
                    print(sample_path + '/' + detail)
                    temp_pert = get_pert_feature(sample_path + '/' + detail)
                    pert_att.append(temp_pert)
                    print(np.array(pert_att).shape)

                if 'roberta_att' in detail:
                    print(sample_path + '/' + detail)
                    temp_roberta = get_roberta_feature(sample_path + '/' + detail)
                    roberta_att.append(temp_roberta)
                    print(np.array(roberta_att).shape)
                
                # if 'graph' in detail:
                #     print(sample_path + '/' + detail)
                #     temp_graph = get_graph(sample_path + '/' + detail)
                #     graph.append(temp_graph)
                #     print(np.array(graph).shape)
                    
        sample_index += 1   
        # face_att = dtw_attention(temp_face, temp_face_vari, temp_iris, temp_index, temp_mfcc, temp_wav2vec, temp_pert, temp_roberta)
        # print(face_att.shape)
        # name = ['face_att']
        # landmarks = pd.DataFrame(columns=name, data=np.array(face_att).reshape(-1,1))
        # landmarks.to_csv(sample_path + '/face_att.csv', encoding='gbk')

        # face_vari_att = dtw_attention(temp_face_vari, temp_face, temp_iris, temp_index, temp_mfcc, temp_wav2vec, temp_pert, temp_roberta)
        # print(face_vari_att.shape)
        # name = ['face_vari_att']
        # landmarks = pd.DataFrame(columns=name, data=np.array(face_vari_att).reshape(-1,1))
        # landmarks.to_csv(sample_path + '/face_vari_att.csv', encoding='gbk')
        
        # iris_att = dtw_attention(temp_iris, temp_face_vari, temp_face, temp_index, temp_mfcc, temp_wav2vec, temp_pert, temp_roberta)
        # print(iris_att.shape)
        # name = ['iris_att']
        # landmarks = pd.DataFrame(columns=name, data=np.array(iris_att).reshape(-1,1))
        # landmarks.to_csv(sample_path + '/iris_att.csv', encoding='gbk')

        # index_att = dtw_attention(temp_index, temp_face_vari, temp_face, temp_iris, temp_mfcc, temp_wav2vec, temp_pert, temp_roberta)
        # print(index_att.shape)
        # name = ['index_att']
        # landmarks = pd.DataFrame(columns=name, data=np.array(index_att).reshape(-1,1))
        # landmarks.to_csv(sample_path + '/index_att.csv', encoding='gbk')

        # mfcc_att = dtw_attention(temp_mfcc, temp_index, temp_face_vari, temp_face, temp_iris, temp_wav2vec, temp_pert, temp_roberta)
        # print(mfcc_att.shape)
        # name = ['mfcc_att']
        # landmarks = pd.DataFrame(columns=name, data=np.array(mfcc_att).reshape(-1,1))
        # landmarks.to_csv(sample_path + '/mfcc_att.csv', encoding='gbk')

        # wav2vec_att = dtw_attention(temp_wav2vec, temp_mfcc, temp_index, temp_face_vari, temp_face, temp_iris, temp_pert, temp_roberta)
        # print(wav2vec_att.shape)
        # name = ['wav2vec_att']
        # landmarks = pd.DataFrame(columns=name, data=np.array(wav2vec_att).reshape(-1,1))
        # landmarks.to_csv(sample_path + '/wav2vec_att.csv', encoding='gbk')

        # pert_att = dtw_attention(temp_pert, temp_wav2vec, temp_mfcc, temp_index, temp_face_vari, temp_face, temp_iris, temp_roberta)
        # print(pert_att.shape)
        # name = ['pert_att']
        # landmarks = pd.DataFrame(columns=name, data=np.array(pert_att).reshape(-1,1))
        # landmarks.to_csv(sample_path + '/pert_att.csv', encoding='gbk')

        # roberta_att = dtw_attention(temp_roberta, temp_pert, temp_wav2vec, temp_mfcc, temp_index, temp_face_vari, temp_face, temp_iris)
        # print(roberta_att.shape)
        # name = ['robera_att']
        # landmarks = pd.DataFrame(columns=name, data=np.array(roberta_att).reshape(-1,1))
        # landmarks.to_csv(sample_path + '/roberta_att.csv', encoding='gbk')
        # print(face_att)


root direction f:\Project\AMH
f:\Project\AMH/muldataset/00002-0101/face_att.csv
(1, 1404)
f:\Project\AMH/muldataset/00002-0101/face_vari_att.csv
(1, 1404)
f:\Project\AMH/muldataset/00002-0101/index_att.csv
(1, 4722)
f:\Project\AMH/muldataset/00002-0101/iris_att.csv
(1, 2776)
f:\Project\AMH/muldataset/00002-0101/mfcc_att.csv
(1, 12987)
f:\Project\AMH/muldataset/00002-0101/pert_att.csv
(1, 1024)
f:\Project\AMH/muldataset/00002-0101/roberta_att.csv
(1, 1024)
f:\Project\AMH/muldataset/00002-0101/wav2vec_att.csv
(1, 1024)
f:\Project\AMH/muldataset/00002-0102/face_att.csv
(2, 1404)
f:\Project\AMH/muldataset/00002-0102/face_vari_att.csv
(2, 1404)
f:\Project\AMH/muldataset/00002-0102/index_att.csv
(2, 4722)
f:\Project\AMH/muldataset/00002-0102/iris_att.csv
(2, 2776)
f:\Project\AMH/muldataset/00002-0102/mfcc_att.csv
(2, 12987)
f:\Project\AMH/muldataset/00002-0102/pert_att.csv
(2, 1024)
f:\Project\AMH/muldataset/00002-0102/roberta_att.csv
(2, 1024)
f:\Project\AMH/muldataset/00002-0102/wav2vec_at

In [368]:
att = []
for i in range(968):
    att.append(face_att[i] + face_vari_att[i] + iris_att[i] + index_att[i] + mfcc_att[i] + wav2vec_att[i] + pert_att[i] + roberta_att[i])


In [369]:
print("Face set:{face}".format(face = np.array(face).shape))
print("Face vari set:{face}".format(face = np.array(face_vari).shape))
print("Iris set:{iris}".format(iris = np.array(iris).shape))
print("Physical index set:{index}".format(index = np.array(physical_index).shape))
print("MFCC set:{mfcc}".format(mfcc = np.array(mfcc_ft).shape))
print("Pert set:{pert}, Roberta:{robert}".format(pert = np.array(pert).shape, \
            robert = np.array(roberta).shape))
print("wav2vec set:{wav2vec}".format(wav2vec = np.array(wav2vec).shape))

print("Att set:{att}".format(att = np.array(att).shape))

print("Graph set:{gra}".format(gra = np.array(graph).shape))
print("Label set:{label}".format(label = np.array(labels).shape)) 

Face set:(968, 1404)
Face vari set:(968, 1404)
Iris set:(968, 2776)
Physical index set:(968, 4722)
MFCC set:(968, 12987)
Pert set:(968, 1024), Roberta:(968, 1024)
wav2vec set:(968, 1024)
Att set:(968, 26365)
Graph set:(968, 64)
Label set:(968,)


In [8]:
def get_face_landmarks(csv_path):
    df = pd.read_csv(csv_path, usecols=[1,2,3])
    # print(df.head(10))
    df_x = df['x']
    df_y = df['y']
    df_z = df['z']
    face_x = df_x.values.tolist()
    face_y = df_y.values.tolist()
    face_z = df_z.values.tolist()

    print("There are {num} faces recorded from this sample.".format(num = np.array(face_x).shape[0] / 468))
    print(np.array(face_x).shape)

    return face_x, face_y, face_z, (np.array(face_x).shape[0] / 468)

def get_avg_face_landmarks(x, y, z, face_num):
    avg_x = []
    avg_y = []
    avg_z = []

    for i in range(468):
        temp = x[i:len(x):468]
        avg_x.append(np.sum(temp)/face_num)
    for i in range(468):
        temp = y[i:len(x):468]
        avg_y.append(np.sum(temp)/face_num)
    for i in range(468):
        temp = z[i:len(x):468]
        avg_z.append(np.sum(temp)/face_num)

    print(np.array(avg_x).shape)
    return avg_x, avg_y, avg_z

def get_vari_face_landmarks(x, y, z, face_num):
    avg_x = []
    avg_y = []
    avg_z = []
    diff = []
    for i in range(468):
        temp = x[i:len(x):468]
        for j in range(len(temp)-1):
            diff.append(abs(temp[j+1] - temp[j]))
        # print(np.array(diff).shape)
        avg_x.append(np.mean(diff))

    diff = []
    for i in range(468):
        temp = y[i:len(y):468]

        for j in range(len(temp)-1):
            diff.append(abs(temp[j+1] - temp[j]))
        avg_y.append(np.mean(diff))
   
    diff = []
    for i in range(468):
        temp = z[i:len(z):468]
        for j in range(len(temp)-1):
            diff.append(abs(temp[j+1] - temp[j]))
        avg_x.append(np.mean(diff))

    print(np.array(avg_x).shape)
    return avg_x, avg_y, avg_z

def get_iris_landmarks(csv_path):
    df = pd.read_csv(csv_path, usecols=[1,2,3,4])
    # print(df.head(10))
    df_leftx = df['left_x']
    df_lefty = df['left_y']
    df_rightx = df['right_x']
    df_righty = df['right_y']
    iris_left_x = df_leftx.values.tolist()
    iris_left_y = df_lefty.values.tolist()
    iris_right_x = df_rightx.values.tolist()
    iris_right_y = df_righty.values.tolist()

    print("There are {num} irises recorded from this sample.".format(num = np.array(df_leftx).shape[0]))
    print(np.array(iris_left_x).shape)

    return iris_left_x, iris_left_y, iris_right_x, iris_right_y, (np.array(df_leftx).shape[0])

def get_avg_iris_landmarks(leftx, lefty, rightx, righty, iris_num):
    avg_leftx = np.sum(leftx) / iris_num
    avg_lefty = np.sum(lefty) / iris_num
    avg_rightx = np.sum(rightx) / iris_num
    avg_righty = np.sum(righty) / iris_num

    return avg_leftx, avg_lefty, avg_rightx, avg_righty

def get_physical_index(csv_path):
    df = pd.read_csv(csv_path, usecols=[1])
    df_index = df['TSfresh'].values.tolist()

    return df_index

def get_voice_feature(csv_path):
    df = pd.read_csv(csv_path, usecols=[1])
    df_mfcc = df['MFCC'].values.tolist()
    
    return df_mfcc

def get_wav2vec(csv_path):
    df = pd.read_csv(csv_path, usecols=[1])
    df_wav2vec = df['wav2vec'].values.tolist()
    
    return df_wav2vec

def get_text_feature(csv_path):
    df = pd.read_csv(csv_path, usecols=[1,2,3])
    df_pert = df['pert'].values.tolist()
    df_roberta = df['roberta'].values.tolist()
    df_macberta = df['macberta'].values.tolist()
    return df_pert, df_roberta, df_macberta

def get_graph_feature(csv_path):
    df = pd.read_csv(csv_path, usecols=[1])
    df_graph = df['graph'].values.tolist()
    
    return df_graph

def get_context_feature(csv_path):
    df = pd.read_csv(csv_path, usecols=[1,2,3])
    df_pert = df['pert'].values.tolist()
    df_roberta = df['roberta'].values.tolist()
    df_macberta = df['macberta'].values.tolist()
    return df_pert, df_roberta, df_macberta

if __name__ == '__main__':
    face = []
    face_vari = []
    iris = []
    physical_index = []
    mfcc_ft = []
    wav2vec = []
    pert = []
    roberta = []
    macberta = []
    # con_pert = []
    # con_roberta = []
    # con_macberta = []
    graph = []
    sample_index = 0

    root = os.path.dirname(os.getcwd())
    print("root direction", root)
    dataset = os.listdir(root + '/muldataset')
    del dataset[-2:]
    for sample in dataset:
        if sample_index in temp:
            sample_index += 1
            continue
        # if '00002' in sample:
        #     continue
        sample_path = root + '/muldataset' + '/' + sample #样本文件夹
        # print(sample_path)
        sample_file = os.listdir(sample_path) #样本文件夹内文件列表
        # print(sample_file)
        for detail in sample_file:
                sample_path = root + '/muldataset' + '/' + sample
                if 'face_landmarks' in detail:
                    # 打开csv文件并读取人脸信息
                    print(sample_path + '/' + detail)
                    face_x, face_y, face_z, face_num = get_face_landmarks(sample_path + '/' + detail)
                    avg_x, avg_y, avg_z = get_avg_face_landmarks(face_x, face_y, face_z, face_num)
                    temp_face = avg_x + avg_y + avg_z
                    face.append(temp_face)
                    print(np.array(face).shape)
                    avg_x, avg_y, avg_z = get_vari_face_landmarks(face_x, face_y, face_z, face_num)
                    temp_face = avg_x + avg_y + avg_z
                    face_vari.append(temp_face)
                    print(np.array(face_vari).shape)
                    

                if 'iris_location' in detail:
                    print(sample_path + '/' + detail)
                    left_x, left_y, right_x, right_y, iris_num = get_iris_landmarks(sample_path + '/' + detail)
                    
                    # avg_leftx, avg_lefty, avg_rightx, avg_righty = get_avg_iris_landmarks(left_x, left_y, right_x, right_y, iris_num)
                    # temp_iris = avg_leftx + avg_lefty + avg_rightx + avg_righty
                    # print(np.array(temp_iris).shape)
                    iris.append(left_x)
                    iris.append(left_y)
                    iris.append(right_x)
                    iris.append(right_y)
                    
                    print(np.array(iris).shape)

                if 'Physical_feature' in detail:
                    print(sample_path + '/' + detail)
                    temp_index = get_physical_index(sample_path + '/' + detail)
                    physical_index.append(temp_index)
                    print(np.array(physical_index).shape)

                if 'mfcc' in detail:
                    print(sample_path + '/' + detail)
                    temp_mfcc = get_voice_feature(sample_path + '/' + detail)
                    mfcc_ft.append(temp_mfcc)
                    print(np.array(mfcc_ft).shape)

                if 'graph' in detail:
                    print(sample_path + '/' + detail)
                    temp_graph = get_graph_feature(sample_path + '/' + detail)
                    graph.append(temp_graph)
                    print(np.array(graph).shape)

                if 'wav2vec' in detail:
                    print(sample_path + '/' + detail)
                    temp_wav2vec = get_wav2vec(sample_path + '/' + detail)
                    wav2vec.append(temp_wav2vec)
                    print(np.array(wav2vec).shape)

                if 'whole_text' in detail:
                    print(sample_path + '/' + detail)
                    temp_pert, temp_roberta, temp_macberta = get_context_feature(sample_path + '/' + detail)
                    pert.append(temp_pert)
                    roberta.append(temp_roberta)
                    macberta.append(temp_macberta)
                    print(np.array(pert).shape, np.array(roberta).shape, np.array(macberta).shape)

                # if 'text_feature' in detail:
                #     print(sample_path + '/' + detail)
                #     temp_pert, temp_roberta, temp_macberta = get_text_feature(sample_path + '/' + detail)
                #     con_pert.append(temp_pert)
                #     con_roberta.append(temp_roberta)
                #     con_macberta.append(temp_macberta)
                #     print(np.array(con_pert).shape, np.array(con_roberta).shape, np.array(con_macberta).shape)

        sample_index += 1        

    iris = np.array(iris).reshape(-1, 694*4)            
    # iris = np.array(iris).reshape(-1, 1)
    print("Face set:{face}".format(face = np.array(face).shape))
    print("Iris set:{iris}".format(iris = np.array(iris).shape))
    print("Physical index set:{index}".format(index = np.array(physical_index).shape))
    print("MFCC set:{mfcc}".format(mfcc = np.array(mfcc_ft).shape))
    print("Wav2vec set:{wav}".format(wav = np.array(wav2vec).shape))
    print("Graph set:{gra}".format(gra = np.array(graph).shape))
    print("Pert set:{pert}, Roberta:{robert} and Macberta:{mac}".format(pert = np.array(pert).shape, \
        robert = np.array(roberta).shape, mac = np.array(macberta).shape))
    # print("concatenate text Pert set:{pert}, Roberta:{robert} and Macberta:{mac}".format(pert = np.array(con_pert).shape, \
    #     robert = np.array(con_roberta).shape, mac = np.array(con_macberta).shape))
    print("Label set:{label}".format(label = np.array(labels).shape))

root direction f:\Project\AMH


NameError: name 'temp' is not defined

In [370]:
from sklearn import preprocessing
skf = StratifiedKFold(n_splits=10)
x = np.array(att)
labels = np.array(labels)
sum_score = []
f1 = []
for train_index, test_index in skf.split(x, labels):
#     print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    # min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0,10))
    # x_train = min_max_scaler.fit_transform(x_train)
    # x_test = min_max_scaler.fit_transform(x_test)

    # scaler = preprocessing.StandardScaler()
    # x_train = scaler.fit_transform(x_train)
    # x_test = scaler.fit_transform(x_test)

    # ros = RandomOverSampler(random_state=0)
    # x_train_resampled, y_train_resampled = ros.fit_resample(x_train, y_train)
    # sm = SMOTE(random_state=0)
    # x_train_resampled, y_train_resampled = sm.fit_resample(x_train, y_train)
    # ada = ADASYN(random_state=0)
    # x_train_resampled, y_train_resampled = ada.fit_resample(x_train, y_train)
    # rus = RandomUnderSampler(random_state=0)
    # x_train_resampled, y_train_resampled = rus.fit_resample(x_train, y_train)
#     smote_enn = SMOTEENN(random_state=0)
#     x_train_resampled, y_train_resampled  = smote_enn.fit_resample(x_train, y_train)
    # smote_tomek = SMOTETomek(random_state=0)
    # x_train_resampled, y_train_resampled = smote_tomek.fit_resample(x_train, y_train)
    # random forest
    RF_clf = RandomForestClassifier(random_state=0, class_weight='balanced')
    # RF_clf.fit(x_train_resampled, y_train_resampled)
    RF_clf.fit(x_train, y_train)
    score_RF = RF_clf.score(x_test, y_test)
    
    y_predicted = RF_clf.predict(x_test)
    test_f1 = f1_score(y_test, y_predicted, average='weighted')
    print(y_predicted)
    print("RF accuracy:{acc}, f1-score:{f1}".format(acc=score_RF, f1=test_f1))
    
    sum_score.append(score_RF)
    f1.append(test_f1)
print("Max accuracy:{max_acc}, F1-score={max_f1}".format(max_acc=np.max(sum_score), max_f1=np.max(f1)))
print("Min accuracy:{min_acc}, F1-score={min_f1}".format(min_acc=np.min(sum_score), min_f1=np.min(f1)))
print("Avg accuracy:{avg_acc}, F1-score={avg_f1}".format(avg_acc=np.mean(sum_score), avg_f1=np.mean(f1)))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
RF accuracy:0.8350515463917526, f1-score:0.7599907332329434
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
RF accuracy:0.8350515463917526, f1-score:0.7599907332329434
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
RF accuracy:0.8350515463917526, f1-score:0.7774653754035197
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0]
RF accuracy:0.85567010309278

In [244]:
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
X = np.array(face)
y = np.array(labels)
total_score = []
y_pred = []
y_true = []
wrong_num_face = []
for train_index, test_index in loo.split(X):
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print("Text index:{ind}".format(ind=test_index))
    RF_clf = RandomForestClassifier(random_state=0, class_weight='balanced')
    RF_clf.fit(x_train, y_train)
    score_RF = RF_clf.score(x_test, y_test)
    total_score.append(score_RF)
    if score_RF == 0:
        wrong_num_face.append(test_index)
    y_pred.append(RF_clf.predict(x_test))
    y_true.append(y_test)
    print("RF accuracy:{acc}, avg socre now is {avg}".format(acc=score_RF, avg = np.mean(total_score)))
test_f1 = f1_score(y_true, y_pred, average='weighted')
print("Score:{score}, F1-score:{f1}".format(score=np.mean(total_score), f1=test_f1))
print(wrong_num_face)

Text index:[0]
RF accuracy:1.0, avg socre now is 1.0
Text index:[1]
RF accuracy:1.0, avg socre now is 1.0
Text index:[2]
RF accuracy:1.0, avg socre now is 1.0
Text index:[3]
RF accuracy:0.0, avg socre now is 0.75
Text index:[4]
RF accuracy:1.0, avg socre now is 0.8
Text index:[5]
RF accuracy:1.0, avg socre now is 0.8333333333333334
Text index:[6]
RF accuracy:1.0, avg socre now is 0.8571428571428571
Text index:[7]
RF accuracy:1.0, avg socre now is 0.875
Text index:[8]
RF accuracy:1.0, avg socre now is 0.8888888888888888
Text index:[9]
RF accuracy:0.0, avg socre now is 0.8
Text index:[10]
RF accuracy:1.0, avg socre now is 0.8181818181818182
Text index:[11]
RF accuracy:1.0, avg socre now is 0.8333333333333334
Text index:[12]
RF accuracy:1.0, avg socre now is 0.8461538461538461
Text index:[13]
RF accuracy:1.0, avg socre now is 0.8571428571428571
Text index:[14]
RF accuracy:1.0, avg socre now is 0.8666666666666667
Text index:[15]
RF accuracy:1.0, avg socre now is 0.875
Text index:[16]
RF ac

In [236]:
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
X = np.array(face_vari)
y = np.array(labels)
total_score = []
y_pred = []
y_true = []
wrong_num_face_vari = []
for train_index, test_index in loo.split(X):
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print("Text index:{ind}".format(ind=test_index))
    RF_clf = RandomForestClassifier(random_state=0, class_weight='balanced')
    RF_clf.fit(x_train, y_train)
    score_RF = RF_clf.score(x_test, y_test)
    total_score.append(score_RF)
    if score_RF == 0:
        wrong_num_face_vari.append(test_index)
    y_pred.append(RF_clf.predict(x_test))
    y_true.append(y_test)
    print("RF accuracy:{acc}, avg socre now is {avg}".format(acc=score_RF, avg = np.mean(total_score)))
test_f1 = f1_score(y_true, y_pred, average='weighted')
print("Score:{score}, F1-score:{f1}".format(score=np.mean(total_score), f1=test_f1))
print(wrong_num_face_vari)

Text index:[0]
RF accuracy:1.0, avg socre now is 1.0
Text index:[1]
RF accuracy:1.0, avg socre now is 1.0
Text index:[2]
RF accuracy:1.0, avg socre now is 1.0
Text index:[3]
RF accuracy:0.0, avg socre now is 0.75
Text index:[4]
RF accuracy:1.0, avg socre now is 0.8
Text index:[5]
RF accuracy:1.0, avg socre now is 0.8333333333333334
Text index:[6]
RF accuracy:1.0, avg socre now is 0.8571428571428571
Text index:[7]
RF accuracy:1.0, avg socre now is 0.875
Text index:[8]
RF accuracy:1.0, avg socre now is 0.8888888888888888
Text index:[9]
RF accuracy:0.0, avg socre now is 0.8
Text index:[10]
RF accuracy:1.0, avg socre now is 0.8181818181818182
Text index:[11]
RF accuracy:1.0, avg socre now is 0.8333333333333334
Text index:[12]
RF accuracy:1.0, avg socre now is 0.8461538461538461
Text index:[13]
RF accuracy:1.0, avg socre now is 0.8571428571428571
Text index:[14]
RF accuracy:1.0, avg socre now is 0.8666666666666667
Text index:[15]
RF accuracy:1.0, avg socre now is 0.875
Text index:[16]
RF ac

In [237]:
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
X = np.array(iris)
y = np.array(labels)
total_score = []
y_pred = []
y_true = []
wrong_num_iris = []
for train_index, test_index in loo.split(X):
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print("Text index:{ind}".format(ind=test_index))
    RF_clf = RandomForestClassifier(random_state=0, class_weight='balanced')
    RF_clf.fit(x_train, y_train)
    score_RF = RF_clf.score(x_test, y_test)
    total_score.append(score_RF)
    if score_RF == 0:
        wrong_num_iris.append(test_index)
    y_pred.append(RF_clf.predict(x_test))
    y_true.append(y_test)
    print("RF accuracy:{acc}, avg socre now is {avg}".format(acc=score_RF, avg = np.mean(total_score)))
test_f1 = f1_score(y_true, y_pred, average='weighted')
print("Score:{score}, F1-score:{f1}".format(score=np.mean(total_score), f1=test_f1))
print(wrong_num_iris)

Text index:[0]
RF accuracy:1.0, avg socre now is 1.0
Text index:[1]
RF accuracy:1.0, avg socre now is 1.0
Text index:[2]
RF accuracy:1.0, avg socre now is 1.0
Text index:[3]
RF accuracy:0.0, avg socre now is 0.75
Text index:[4]
RF accuracy:1.0, avg socre now is 0.8
Text index:[5]
RF accuracy:1.0, avg socre now is 0.8333333333333334
Text index:[6]
RF accuracy:1.0, avg socre now is 0.8571428571428571
Text index:[7]
RF accuracy:1.0, avg socre now is 0.875
Text index:[8]
RF accuracy:1.0, avg socre now is 0.8888888888888888
Text index:[9]
RF accuracy:0.0, avg socre now is 0.8
Text index:[10]
RF accuracy:1.0, avg socre now is 0.8181818181818182
Text index:[11]
RF accuracy:1.0, avg socre now is 0.8333333333333334
Text index:[12]
RF accuracy:1.0, avg socre now is 0.8461538461538461
Text index:[13]
RF accuracy:1.0, avg socre now is 0.8571428571428571
Text index:[14]
RF accuracy:1.0, avg socre now is 0.8666666666666667
Text index:[15]
RF accuracy:1.0, avg socre now is 0.875
Text index:[16]
RF ac

In [238]:
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
X = np.array(physical_index)
y = np.array(labels)
total_score = []
y_pred = []
y_true = []
wrong_num_phy = []
for train_index, test_index in loo.split(X):
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print("Text index:{ind}".format(ind=test_index))
    RF_clf = RandomForestClassifier(random_state=0, class_weight='balanced')
    RF_clf.fit(x_train, y_train)
    score_RF = RF_clf.score(x_test, y_test)
    total_score.append(score_RF)
    if score_RF == 0:
        wrong_num_phy.append(test_index)
    y_pred.append(RF_clf.predict(x_test))
    y_true.append(y_test)
    print("RF accuracy:{acc}, avg socre now is {avg}".format(acc=score_RF, avg = np.mean(total_score)))
test_f1 = f1_score(y_true, y_pred, average='weighted')
print("Score:{score}, F1-score:{f1}".format(score=np.mean(total_score), f1=test_f1))
print(wrong_num_phy)

Text index:[0]
RF accuracy:1.0, avg socre now is 1.0
Text index:[1]
RF accuracy:1.0, avg socre now is 1.0
Text index:[2]
RF accuracy:1.0, avg socre now is 1.0
Text index:[3]
RF accuracy:0.0, avg socre now is 0.75
Text index:[4]
RF accuracy:1.0, avg socre now is 0.8
Text index:[5]
RF accuracy:1.0, avg socre now is 0.8333333333333334
Text index:[6]
RF accuracy:1.0, avg socre now is 0.8571428571428571
Text index:[7]
RF accuracy:1.0, avg socre now is 0.875
Text index:[8]
RF accuracy:1.0, avg socre now is 0.8888888888888888
Text index:[9]
RF accuracy:0.0, avg socre now is 0.8
Text index:[10]
RF accuracy:1.0, avg socre now is 0.8181818181818182
Text index:[11]
RF accuracy:1.0, avg socre now is 0.8333333333333334
Text index:[12]
RF accuracy:1.0, avg socre now is 0.8461538461538461
Text index:[13]
RF accuracy:1.0, avg socre now is 0.8571428571428571
Text index:[14]
RF accuracy:1.0, avg socre now is 0.8666666666666667
Text index:[15]
RF accuracy:1.0, avg socre now is 0.875
Text index:[16]
RF ac

In [239]:
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
X = np.array(mfcc_ft)
y = np.array(labels)
total_score = []
y_pred = []
y_true = []
wrong_num_mfcc = []
for train_index, test_index in loo.split(X):
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print("Text index:{ind}".format(ind=test_index))
    RF_clf = RandomForestClassifier(random_state=0, class_weight='balanced')
    RF_clf.fit(x_train, y_train)
    score_RF = RF_clf.score(x_test, y_test)
    total_score.append(score_RF)
    if score_RF == 0:
        wrong_num_mfcc.append(test_index)
    y_pred.append(RF_clf.predict(x_test))
    y_true.append(y_test)
    print("RF accuracy:{acc}, avg socre now is {avg}".format(acc=score_RF, avg = np.mean(total_score)))
test_f1 = f1_score(y_true, y_pred, average='weighted')
print("Score:{score}, F1-score:{f1}".format(score=np.mean(total_score), f1=test_f1))
print(wrong_num_mfcc)

Text index:[0]
RF accuracy:1.0, avg socre now is 1.0
Text index:[1]
RF accuracy:1.0, avg socre now is 1.0
Text index:[2]
RF accuracy:1.0, avg socre now is 1.0
Text index:[3]
RF accuracy:0.0, avg socre now is 0.75
Text index:[4]
RF accuracy:1.0, avg socre now is 0.8
Text index:[5]
RF accuracy:1.0, avg socre now is 0.8333333333333334
Text index:[6]
RF accuracy:1.0, avg socre now is 0.8571428571428571
Text index:[7]
RF accuracy:1.0, avg socre now is 0.875
Text index:[8]
RF accuracy:1.0, avg socre now is 0.8888888888888888
Text index:[9]
RF accuracy:0.0, avg socre now is 0.8
Text index:[10]
RF accuracy:1.0, avg socre now is 0.8181818181818182
Text index:[11]
RF accuracy:1.0, avg socre now is 0.8333333333333334
Text index:[12]
RF accuracy:1.0, avg socre now is 0.8461538461538461
Text index:[13]
RF accuracy:1.0, avg socre now is 0.8571428571428571
Text index:[14]
RF accuracy:1.0, avg socre now is 0.8666666666666667
Text index:[15]
RF accuracy:1.0, avg socre now is 0.875
Text index:[16]
RF ac

In [240]:
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
X = np.array(wav2vec)
y = np.array(labels)
total_score = []
y_pred = []
y_true = []
wrong_num_wav = []
for train_index, test_index in loo.split(X):
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print("Text index:{ind}".format(ind=test_index))
    RF_clf = RandomForestClassifier(random_state=0, class_weight='balanced')
    RF_clf.fit(x_train, y_train)
    score_RF = RF_clf.score(x_test, y_test)
    total_score.append(score_RF)
    if score_RF == 0:
        wrong_num_wav.append(test_index)
    y_pred.append(RF_clf.predict(x_test))
    y_true.append(y_test)
    print("RF accuracy:{acc}, avg socre now is {avg}".format(acc=score_RF, avg = np.mean(total_score)))
test_f1 = f1_score(y_true, y_pred, average='weighted')
print("Score:{score}, F1-score:{f1}".format(score=np.mean(total_score), f1=test_f1))
print(wrong_num_wav)

Text index:[0]
RF accuracy:1.0, avg socre now is 1.0
Text index:[1]
RF accuracy:1.0, avg socre now is 1.0
Text index:[2]
RF accuracy:1.0, avg socre now is 1.0
Text index:[3]
RF accuracy:0.0, avg socre now is 0.75
Text index:[4]
RF accuracy:1.0, avg socre now is 0.8
Text index:[5]
RF accuracy:1.0, avg socre now is 0.8333333333333334
Text index:[6]
RF accuracy:1.0, avg socre now is 0.8571428571428571
Text index:[7]
RF accuracy:1.0, avg socre now is 0.875
Text index:[8]
RF accuracy:1.0, avg socre now is 0.8888888888888888
Text index:[9]
RF accuracy:0.0, avg socre now is 0.8
Text index:[10]
RF accuracy:1.0, avg socre now is 0.8181818181818182
Text index:[11]
RF accuracy:1.0, avg socre now is 0.8333333333333334
Text index:[12]
RF accuracy:1.0, avg socre now is 0.8461538461538461
Text index:[13]
RF accuracy:1.0, avg socre now is 0.8571428571428571
Text index:[14]
RF accuracy:1.0, avg socre now is 0.8666666666666667
Text index:[15]
RF accuracy:1.0, avg socre now is 0.875
Text index:[16]
RF ac

In [241]:
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
X = np.array(roberta)
y = np.array(labels)
total_score = []
y_pred = []
y_true = []
wrong_num_rob = []
for train_index, test_index in loo.split(X):
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print("Text index:{ind}".format(ind=test_index))
    RF_clf = RandomForestClassifier(random_state=0, class_weight='balanced')
    RF_clf.fit(x_train, y_train)
    score_RF = RF_clf.score(x_test, y_test)
    total_score.append(score_RF)
    if score_RF == 0:
        wrong_num_rob.append(test_index)
    y_pred.append(RF_clf.predict(x_test))
    y_true.append(y_test)
    print("RF accuracy:{acc}, avg socre now is {avg}".format(acc=score_RF, avg = np.mean(total_score)))
test_f1 = f1_score(y_true, y_pred, average='weighted')
print("Score:{score}, F1-score:{f1}".format(score=np.mean(total_score), f1=test_f1))
print(wrong_num_rob)

Text index:[0]
RF accuracy:1.0, avg socre now is 1.0
Text index:[1]
RF accuracy:1.0, avg socre now is 1.0
Text index:[2]
RF accuracy:1.0, avg socre now is 1.0
Text index:[3]
RF accuracy:0.0, avg socre now is 0.75
Text index:[4]
RF accuracy:1.0, avg socre now is 0.8
Text index:[5]
RF accuracy:1.0, avg socre now is 0.8333333333333334
Text index:[6]
RF accuracy:1.0, avg socre now is 0.8571428571428571
Text index:[7]
RF accuracy:1.0, avg socre now is 0.875
Text index:[8]
RF accuracy:1.0, avg socre now is 0.8888888888888888
Text index:[9]
RF accuracy:0.0, avg socre now is 0.8
Text index:[10]
RF accuracy:1.0, avg socre now is 0.8181818181818182
Text index:[11]
RF accuracy:1.0, avg socre now is 0.8333333333333334
Text index:[12]
RF accuracy:1.0, avg socre now is 0.8461538461538461
Text index:[13]
RF accuracy:1.0, avg socre now is 0.8571428571428571
Text index:[14]
RF accuracy:1.0, avg socre now is 0.8666666666666667
Text index:[15]
RF accuracy:1.0, avg socre now is 0.875
Text index:[16]
RF ac

In [242]:
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
X = np.array(pert)
y = np.array(labels)
total_score = []
y_pred = []
y_true = []
wrong_num_pert = []
for train_index, test_index in loo.split(X):
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print("Text index:{ind}".format(ind=test_index))
    RF_clf = RandomForestClassifier(random_state=0, class_weight='balanced')
    RF_clf.fit(x_train, y_train)
    score_RF = RF_clf.score(x_test, y_test)
    total_score.append(score_RF)
    if score_RF == 0:
        wrong_num_pert.append(test_index)
    y_pred.append(RF_clf.predict(x_test))
    y_true.append(y_test)
    print("RF accuracy:{acc}, avg socre now is {avg}".format(acc=score_RF, avg = np.mean(total_score)))
test_f1 = f1_score(y_true, y_pred, average='weighted')
print("Score:{score}, F1-score:{f1}".format(score=np.mean(total_score), f1=test_f1))
print(wrong_num_pert)

Text index:[0]
RF accuracy:1.0, avg socre now is 1.0
Text index:[1]
RF accuracy:1.0, avg socre now is 1.0
Text index:[2]
RF accuracy:1.0, avg socre now is 1.0
Text index:[3]
RF accuracy:0.0, avg socre now is 0.75
Text index:[4]
RF accuracy:1.0, avg socre now is 0.8
Text index:[5]
RF accuracy:1.0, avg socre now is 0.8333333333333334
Text index:[6]
RF accuracy:1.0, avg socre now is 0.8571428571428571
Text index:[7]
RF accuracy:1.0, avg socre now is 0.875
Text index:[8]
RF accuracy:1.0, avg socre now is 0.8888888888888888
Text index:[9]
RF accuracy:0.0, avg socre now is 0.8
Text index:[10]
RF accuracy:1.0, avg socre now is 0.8181818181818182
Text index:[11]
RF accuracy:1.0, avg socre now is 0.8333333333333334
Text index:[12]
RF accuracy:1.0, avg socre now is 0.8461538461538461
Text index:[13]
RF accuracy:1.0, avg socre now is 0.8571428571428571
Text index:[14]
RF accuracy:1.0, avg socre now is 0.8666666666666667
Text index:[15]
RF accuracy:1.0, avg socre now is 0.875
Text index:[16]
RF ac

In [254]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # model = EmbraceTextAudio(device, [768, 512], 256)
# model = EmbraceTextAudioVideo(device, [1024, 9828, 1404, 4, 4722], 4096)

In [168]:
iris = np.array(iris).reshape(-1, 694*4)         
print("Face set:{face}".format(face = np.array(face).shape))
print("Face vari set:{face}".format(face = np.array(face_vari).shape))
print("Iris set:{iris}".format(iris = np.array(iris).shape))
print("Physical index set:{index}".format(index = np.array(physical_index).shape))
print("MFCC set:{mfcc}".format(mfcc = np.array(mfcc_ft).shape))
print("Pert set:{pert}, Roberta:{robert} and Macberta:{mac}".format(pert = np.array(pert).shape, \
        robert = np.array(roberta).shape, mac = np.array(macberta).shape))
print("wav2vec set:{wav2vec}".format(wav2vec = np.array(wav2vec).shape))
# print("Graph set:{gra}".format(gra = np.array(graph).shape))
print("Label set:{label}".format(label = np.array(labels).shape))

Face set:(1011, 1404)
Face vari set:(1011, 1404)
Iris set:(1011, 2776)
Physical index set:(1011, 4722)
MFCC set:(1011, 12987)
Pert set:(1011, 1024), Roberta:(1011, 1024) and Macberta:(1011, 1024)
wav2vec set:(1011, 1024)
Label set:(1011,)


In [374]:
from sklearn import preprocessing
from sklearn import decomposition

skf = StratifiedKFold(n_splits=10)
min_max_scaler = preprocessing.MinMaxScaler()
scaler = preprocessing.StandardScaler()
pca = decomposition.PCA(n_components=1024)
y = labels

x = np.array(face)
y = np.array(y)
face_x_train_skf = []
face_x_test_skf = []
face_y_train_skf = []
face_y_test_skf = []
for train_index, test_index in skf.split(x, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # x_train = min_max_scaler.fit_transform(x_train)
    # x_test = min_max_scaler.fit_transform(x_test)
    # for i in range(0, 375):
    #     x_train[i] = filter_signal(x_train[i], threshold=1e3)
    # for i in range(0, 41):
    #     x_test[i] = filter_signal(x_test[i], threshold=1e3)
    # smoother.smooth(x_train)
    # x_train = smoother.smooth_data
    # smoother.smooth(x_test)
    # x_test = smoother.smooth_data
    # x_train = pca.fit_transform(x_train)
    # x_test = pca.fit_transform(x_test)
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.fit_transform(x_test)
    # x_train = preprocessing.scale(x_train)
    # x_test = preprocessing.scale(x_test)
    face_x_train_skf.append(x_train)
    face_x_test_skf.append(x_test)
    face_y_train_skf.append(y_train)
    face_y_test_skf.append(y_test)
print("face shape:", np.array(face_x_train_skf[0]).shape)
print("face shape:", np.array(face_x_test_skf[0]).shape)

x = np.array(face_vari)
y = np.array(y)
face_vari_x_train_skf = []
face_vari_x_test_skf = []
face_vari_y_train_skf = []
face_vari_y_test_skf = []
for train_index, test_index in skf.split(x, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # x_train = min_max_scaler.fit_transform(x_train)
    # x_test = min_max_scaler.fit_transform(x_test)
    # for i in range(0, 375):
    #     x_train[i] = filter_signal(x_train[i], threshold=1e3)
    # for i in range(0, 41):
    #     x_test[i] = filter_signal(x_test[i], threshold=1e3)
    # smoother.smooth(x_train)
    # x_train = smoother.smooth_data
    # smoother.smooth(x_test)
    # x_test = smoother.smooth_data
    # x_train = pca.fit_transform(x_train)
    # x_test = pca.fit_transform(x_test)
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.fit_transform(x_test)
    # x_train = preprocessing.scale(x_train)
    # x_test = preprocessing.scale(x_test)
    face_vari_x_train_skf.append(x_train)
    face_vari_x_test_skf.append(x_test)
    face_vari_y_train_skf.append(y_train)
    face_vari_y_test_skf.append(y_test)
print("face vari shape:", np.array(face_vari_x_train_skf[0]).shape)
print("face vari shape:", np.array(face_vari_x_test_skf[0]).shape)

x = np.array(iris)
y = np.array(y)
iris_x_train_skf = []
iris_x_test_skf = []
iris_y_train_skf = []
iris_y_test_skf = []
for train_index, test_index in skf.split(x, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # x_train = min_max_scaler.fit_transform(x_train)
    # x_test = min_max_scaler.fit_transform(x_test)
    # for i in range(0, 375):
    #     x_train[i] = filter_signal(x_train[i], threshold=1e3)
    # for i in range(0, 41):
    #     x_test[i] = filter_signal(x_test[i], threshold=1e3)
    # x_train = pca.fit_transform(x_train)
    # x_test = pca.fit_transform(x_test)
    # x_train = preprocessing.scale(x_train)
    # x_test = preprocessing.scale(x_test)
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.fit_transform(x_test)
    iris_x_train_skf.append(x_train)
    iris_x_test_skf.append(x_test)
    iris_y_train_skf.append(y_train)
    iris_y_test_skf.append(y_test)
print("iris shape:", np.array(iris_x_train_skf[0]).shape)
print("iris shape:", np.array(iris_x_test_skf[0]).shape)

x = np.array(physical_index)
y = np.array(y)
physical_x_train_skf = []
physical_x_test_skf = []
physical_y_train_skf = []
physical_y_test_skf = []
for train_index, test_index in skf.split(x, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # x_train = min_max_scaler.fit_transform(x_train)
    # x_test = min_max_scaler.fit_transform(x_test)
    # for i in range(0, 375):
    #     x_train[i] = filter_signal(x_train[i], threshold=1e3)
    # for i in range(0, 41):
    #     x_test[i] = filter_signal(x_test[i], threshold=1e3)
    # smoother.smooth(x_train)
    # x_train = smoother.smooth_data
    # smoother.smooth(x_test)
    # x_test = smoother.smooth_data
    # x_train = pca.fit_transform(x_train)
    # x_test = pca.fit_transform(x_test)
    # x_train = preprocessing.scale(x_train)
    # x_test = preprocessing.scale(x_test)
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.fit_transform(x_test)
    physical_x_train_skf.append(x_train)
    physical_x_test_skf.append(x_test)
    physical_y_train_skf.append(y_train)
    physical_y_test_skf.append(y_test)
print("physical index shape:", np.array(physical_x_train_skf[0]).shape)
print("physical index shape:", np.array(physical_x_test_skf[0]).shape)

x = np.array(mfcc_ft)
y = np.array(y)
mfcc_ft_x_train_skf = []
mfcc_ft_x_test_skf = []
mfcc_ft_y_train_skf = []
mfcc_ft_y_test_skf = []
for train_index, test_index in skf.split(x, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # x_train = min_max_scaler.fit_transform(x_train)
    # x_test = min_max_scaler.fit_transform(x_test)
    # for i in range(0, 375):
    #     x_train[i] = filter_signal(x_train[i], threshold=1e3)
    # for i in range(0, 41):
    #     x_test[i] = filter_signal(x_test[i], threshold=1e3)
    # smoother.smooth(x_train)
    # x_train = smoother.smooth_data
    # smoother.smooth(x_test)
    # x_test = smoother.smooth_data
    # x_train = pca.fit_transform(x_train)
    # x_test = pca.fit_transform(x_test)
    # x_train = preprocessing.scale(x_train)
    # x_test = preprocessing.scale(x_test)
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.fit_transform(x_test)
    mfcc_ft_x_train_skf.append(x_train)
    mfcc_ft_x_test_skf.append(x_test)  
    mfcc_ft_y_train_skf.append(y_train)
    mfcc_ft_y_test_skf.append(y_test)
print("mfcc shape:", np.array(mfcc_ft_x_train_skf[0]).shape)
print("mfcc shape:", np.array(mfcc_ft_x_test_skf[0]).shape)

x = np.array(wav2vec)
y = np.array(y)
wav2vec_x_train_skf = []
wav2vec_x_test_skf = []
wav2vec_y_train_skf = []
wav2vec_y_test_skf = []
for train_index, test_index in skf.split(x, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # x_train = min_max_scaler.fit_transform(x_train)
    # x_test = min_max_scaler.fit_transform(x_test)
    # for i in range(0, 375):
    #     x_train[i] = filter_signal(x_train[i], threshold=1e3)
    # for i in range(0, 41):
    #     x_test[i] = filter_signal(x_test[i], threshold=1e3)
    # smoother.smooth(x_train)
    # x_train = smoother.smooth_data
    # smoother.smooth(x_test)
    # x_test = smoother.smooth_data
    # x_train = pca.fit_transform(x_train)
    # x_test = pca.fit_transform(x_test)
    # x_train = preprocessing.scale(x_train)
    # x_test = preprocessing.scale(x_test)
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.fit_transform(x_test)
    wav2vec_x_train_skf.append(x_train)
    wav2vec_x_test_skf.append(x_test)  
    wav2vec_y_train_skf.append(y_train)
    wav2vec_y_test_skf.append(y_test)
print("wav2vec shape:", np.array(wav2vec_x_train_skf[0]).shape)
print("wav2vec shape:", np.array(wav2vec_x_test_skf[0]).shape)

x = np.array(pert)
y = np.array(y)
pert_x_train_skf = []
pert_x_test_skf = []
pert_y_train_skf = []
pert_y_test_skf = []
for train_index, test_index in skf.split(x, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # x_train = min_max_scaler.fit_transform(x_train)
    # x_test = min_max_scaler.fit_transform(x_test)
    # for i in range(0, 375):
    #     x_train[i] = filter_signal(x_train[i], threshold=1e3)
    # for i in range(0, 41):
    #     x_test[i] = filter_signal(x_test[i], threshold=1e3)
    
    # smoother.smooth(x_train)
    # x_train = smoother.smooth_data
    # smoother.smooth(x_test)
    # x_test = smoother.smooth_data
    # x_train = pca.fit_transform(x_train)
    # x_test = pca.fit_transform(x_test)
    # x_train = preprocessing.scale(x_train)
    # x_test = preprocessing.scale(x_test)
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.fit_transform(x_test)
    pert_x_train_skf.append(x_train)
    pert_x_test_skf.append(x_test)
    pert_y_train_skf.append(y_train)
    pert_y_test_skf.append(y_test)
print("pert shape:", np.array(pert_x_train_skf[0]).shape)
print("pert shape:", np.array(pert_x_test_skf[0]).shape)

x = np.array(roberta)
y = np.array(y)
roberta_x_train_skf = []
roberta_x_test_skf = []
roberta_y_train_skf = []
roberta_y_test_skf = []
for train_index, test_index in skf.split(x, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # x_train = min_max_scaler.fit_transform(x_train)
    # x_test = min_max_scaler.fit_transform(x_test)
    # for i in range(0, 375):
    #     x_train[i] = filter_signal(x_train[i], threshold=1e3)
    # for i in range(0, 41):
    #     x_test[i] = filter_signal(x_test[i], threshold=1e3)
    # smoother.smooth(x_train)
    # x_train = smoother.smooth_data
    # smoother.smooth(x_test)
    # x_test = smoother.smooth_data
    # x_train = pca.fit_transform(x_train)
    # x_test = pca.fit_transform(x_test)
    # x_train = preprocessing.scale(x_train)
    # x_test = preprocessing.scale(x_test)
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.fit_transform(x_test)
    roberta_x_train_skf.append(x_train)
    roberta_x_test_skf.append(x_test)
    roberta_y_train_skf.append(y_train)
    roberta_y_test_skf.append(y_test)
print("roberta shape:", np.array(roberta_x_train_skf[0]).shape)
print("roberta shape:", np.array(roberta_x_test_skf[0]).shape)

x = np.array(att)
y = np.array(y)
att_x_train_skf = []
att_x_test_skf = []
att_y_train_skf = []
att_y_test_skf = []
for train_index, test_index in skf.split(x, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # x_train = min_max_scaler.fit_transform(x_train)
    # x_test = min_max_scaler.fit_transform(x_test)

    # for i in range(0, 375):
    #     x_train[i] = filter_signal(x_train[i], threshold=1e3)
    # for i in range(0, 41):
    #     x_test[i] = filter_signal(x_test[i], threshold=1e3)
    # smoother.smooth(x_train)
    # x_train = smoother.smooth_data
    # smoother.smooth(x_test)
    # x_test = smoother.smooth_data

    # x_train = pca.fit_transform(x_train)
    # x_test = pca.fit_transform(x_test)
    # x_train = preprocessing.scale(x_train)
    # x_test = preprocessing.scale(x_test)
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.fit_transform(x_test)
    att_x_train_skf.append(x_train)
    att_x_test_skf.append(x_test)
    att_y_train_skf.append(y_train)
    att_y_test_skf.append(y_test)
print("att shape:", np.array(att_x_train_skf[0]).shape)
print("att shape:", np.array(att_x_test_skf[0]).shape)

x = np.array(graph)
y = np.array(y)
graph_x_train_skf = []
graph_x_test_skf = []
graph_y_train_skf = []
graph_y_test_skf = []
for train_index, test_index in skf.split(x, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    x_train = min_max_scaler.fit_transform(x_train)
    x_test = min_max_scaler.fit_transform(x_test)

    # for i in range(0, 375):
    #     x_train[i] = filter_signal(x_train[i], threshold=1e3)
    # for i in range(0, 41):
    #     x_test[i] = filter_signal(x_test[i], threshold=1e3)
    # smoother.smooth(x_train)
    # x_train = smoother.smooth_data
    # smoother.smooth(x_test)
    # x_test = smoother.smooth_data

    # x_train = pca.fit_transform(x_train)
    # x_test = pca.fit_transform(x_test)
    # x_train = preprocessing.scale(x_train)
    # x_test = preprocessing.scale(x_test)
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.fit_transform(x_test)
    graph_x_train_skf.append(x_train)
    graph_x_test_skf.append(x_test)
    graph_y_train_skf.append(y_train)
    graph_y_test_skf.append(y_test)
print("graph shape:", np.array(graph_x_train_skf[0]).shape)
print("graph shape:", np.array(graph_x_test_skf[0]).shape)



face shape: (871, 1404)
face shape: (97, 1404)
face vari shape: (871, 1404)
face vari shape: (97, 1404)
iris shape: (871, 2776)
iris shape: (97, 2776)
physical index shape: (871, 4722)
physical index shape: (97, 4722)
mfcc shape: (871, 12987)
mfcc shape: (97, 12987)
wav2vec shape: (871, 1024)
wav2vec shape: (97, 1024)
pert shape: (871, 1024)
pert shape: (97, 1024)
roberta shape: (871, 1024)
roberta shape: (97, 1024)
att shape: (871, 26365)
att shape: (97, 26365)
graph shape: (871, 64)
graph shape: (97, 64)


In [219]:
from dtaidistance import dtw
def softmax(x):
    x = x - np.max(x)
    exp_x = np.exp(x)
    softmax_x = exp_x / np.sum(exp_x)
    return softmax_x

def dtw_attention(v1, v2, v3, v4, v5, v6, v7, v8, v9):
    v1 = np.array(v1)
    v2 = np.array(v2)
    v3 = np.array(v3)
    v4 = np.array(v4)
    v5 = np.array(v5)
    v6 = np.array(v6)
    v7 = np.array(v7)
    v8 = np.array(v8)
    v9 = np.array(v9)

    w12 = dtw.distance_fast(v1, v2)
    w13 = dtw.distance_fast(v1, v3)
    w14 = dtw.distance_fast(v1, v4)
    w15 = dtw.distance_fast(v1, v5)
    w16 = dtw.distance_fast(v1, v6)
    w17 = dtw.distance_fast(v1, v7)
    w18 = dtw.distance_fast(v1, v8)
    w19 = dtw.distance_fast(v1, v9)

    w = [1/w12, 1/w13, 1/w14, 1/w15, 1/w16, 1/w17, 1/w18, 1/w19]
    print(w)
    w = softmax(w)
    print(w)
    
    output = v1.tolist() + (w12*v2).tolist() + (w13*v3).tolist() + (w14*v4).tolist() + (w15*v5).tolist() + (w16*v6).tolist() + (w17*v7).tolist() + (w18*v8).tolist() + (w19*v9).tolist()
    print(np.array(output).shape)
    return output

output = dtw_attention(face[0], face_vari[0], iris[0], physical_index[0], mfcc_ft[0], wav2vec[0], pert[0], roberta[0],  macberta[0])
print(output)

[0.06108665734182983, 0.12429679791397893, 5.94804786240166e-07, 0.00108679585791488, 0.1107776368225825, 0.11634855192504528, 0.08693633805783835, 0.097554004450378]
[0.12317018 0.13120711 0.11587139 0.11599732 0.12944524 0.13016838
 0.1263956  0.12774477]
(27389,)
[0.5149595495664848, 0.5163439915708776, 0.5147765760995308, 0.4694372814781261, 0.5139053538441658, 0.5088821216574255, 0.49469715666096165, 0.20636022006565669, 0.4834140768309809, 0.47938722554805147, 0.4668824370458441, 0.5159636062552344, 0.5169096645319237, 0.5177968043201374, 0.5186622617941983, 0.5202206166566543, 0.5217210246988062, 0.5233118642332419, 0.5262096931911865, 0.5165600986132082, 0.4754031515627537, 0.04952237122463726, 0.3141421248328011, 0.2784041314614269, 0.24239370789168016, 0.19070559016095018, 0.3442449580948308, 0.25150173558097966, 0.2952248269938073, 0.21083966698848977, 0.18462012912023743, 0.16102444977973993, 0.3860308179315531, 0.1904047925095513, 0.04668207058929044, 0.11607268431557796, 

In [216]:
files = os.listdir('./checkpoint/final/')
i = 0
final_acc = []
for detail in files:
        pt_name = 'Round_' + str(i)
        if pt_name in detail:
            num = detail[-8:-4]
            print(int(num)/100)
            final_acc.append(int(num)/100)
        i += 1
print(np.mean(final_acc))

88.66
87.63
87.63
90.72
87.63
88.66
87.63
89.69
88.54
89.58
88.637


In [383]:
class EmbraceNet(nn.Module):
  
  def __init__(self, device, input_size_list, embracement_size=256, bypass_docking=False):
    """
    Initialize an EmbraceNet module.
    Args:
      device: A "torch.device()" object to allocate internal parameters of the EmbraceNet module.
      input_size_list: A list of input sizes.
      embracement_size: The length of the output of the embracement layer ("c" in the paper).
      bypass_docking: Bypass docking step, i.e., connect the input data directly to the embracement layer. If True, input_data must have a shape of [batch_size, embracement_size].
    """
    super(EmbraceNet, self).__init__()

    self.device = device
    self.input_size_list = input_size_list
    self.embracement_size = embracement_size
    self.bypass_docking = bypass_docking
    # self.drop = nn.Dropout(p=0.5)
  
    if (not bypass_docking):
      for i, input_size in enumerate(input_size_list):
        # setattr(self, 'project_%d' % (i), nn.Linear(input_size, embracement_size * 4))
        # setattr(self, 'docking_%d' % (i), nn.Linear(embracement_size* 4 , embracement_size))
        setattr(self, 'docking_%d' % (i), nn.Linear(input_size , embracement_size))
 
  def forward(self, input_list, availabilities=None, selection_probabilities=None):
    """
    Forward input data to the EmbraceNet module.
    Args:
      input_list: A list of input data. Each input data should have a size as in input_size_list.
      availabilities: A 2-D tensor of shape [batch_size, num_modalities], which represents the availability of data for each modality. If None, it assumes that data of all modalities are available.
      selection_probabilities: A 2-D tensor of shape [batch_size, num_modalities], which represents probabilities that output of each docking layer will be selected ("p" in the paper). If None, the same probability of being selected will be used for each docking layer.
    Returns:
      A 2-D tensor of shape [batch_size, embracement_size] that is the embraced output.
    """

    # check input data
    assert len(input_list) == len(self.input_size_list)
    num_modalities = len(input_list)
    batch_size = input_list[0].shape[0]
    
    
    # docking layer
    docking_output_list = []
    distance = []
    if (self.bypass_docking):
      docking_output_list = input_list
    else:
      for i, input_data in enumerate(input_list):
        # x = getattr(self, 'project_%d' % (i))(input_data)
        x = getattr(self, 'docking_%d' % (i))(input_data)
        # x = self.drop(x)
        # dropped = self.drop(x)
      
        x = nn.functional.relu(x)
        docking_output_list.append(x)
     
    # check availabilities
    if (availabilities is None):
      availabilities = torch.ones(batch_size, len(input_list), dtype=torch.float, device=self.device)
    else:
      availabilities = availabilities.float()
    
    # adjust selection probabilities
    if (selection_probabilities is None):
      selection_probabilities = torch.ones(batch_size, len(input_list), dtype=torch.float, device=self.device)
    selection_probabilities = torch.mul(selection_probabilities, availabilities)
    
    probability_sum = torch.sum(selection_probabilities, dim=-1, keepdim=True)
    selection_probabilities = torch.div(selection_probabilities, probability_sum)
    # stack docking outputs
    docking_output_stack = torch.stack(docking_output_list, dim=-1)  # [batch_size, embracement_size, num_modalities]

    # embrace
    modality_indices = torch.multinomial(selection_probabilities, num_samples=self.embracement_size, replacement=True)  # [batch_size, embracement_size]
#     print(modality_indices)
    modality_toggles = nn.functional.one_hot(modality_indices, num_classes=num_modalities).float()  # [batch_size, embracement_size, num_modalities]

    embracement_output_stack = torch.mul(docking_output_stack, modality_toggles)
    embracement_output = torch.sum(embracement_output_stack, dim=-1)  # [batch_size, embracement_size]
    return embracement_output, modality_indices


class EmbraceTextAudioVideo(nn.Module):
    def __init__(self, device, input_size, embrace_size):
        super(EmbraceTextAudioVideo, self).__init__()

        self.device = device
        # embracenet
        self.embracenet = EmbraceNet(device=self.device, input_size_list=input_size, embracement_size=embrace_size)

        # post embracement layers
        self.post = nn.Linear(in_features=embrace_size, out_features=2)

        # self.acti = nn.Tanhshrink()
        # self.acti = nn.Hardtanh()
        # self.acti = nn.ELU()
        # self.acti = nn.Softsign()
        # self.acti = nn.LogSigmoid()
        # self.acti = nn.Tanh()

    def forward(self, roberta, pert, wav, wav2vec, iris, physical_index, face, face_vari, att, graph):
        embraced_output, modality_indices = self.embracenet(input_list=[face, face_vari, iris, wav, wav2vec, physical_index, roberta, pert, att, graph])
        # embraced_output = self.embracenet(input_list=[roberta, pert, macbert, wav, iris, physical_index])
        x = self.post(embraced_output)
        # print(x)
        x = torch.sigmoid(x)
        # x = torch.relu(x)
        # x = torch.rrelu(x)
        # x = torch.selu(x)
        # x = torch.celu(x)
        # x = self.acti(x)
        return x, modality_indices


total_acc = []
total_f1 = []

if torch.cuda.is_available():
        dtype = torch.cuda.FloatTensor
        ltype = torch.cuda.LongTensor
else:
        dtype = torch.FloatTensor
        ltype = torch.LongTensor

for j in range(0, 10):
        ACC = []
        F1 = []
        for i in range(0, 10):
                print("The {round} group of {time}".format(round=i, time = j))
                # if i < 8:
                #         continue
                face_x_train, face_y_train = face_x_train_skf[i], face_y_train_skf[i]
                face_vari_x_train, face_vari_y_train = face_vari_x_train_skf[i], face_vari_y_train_skf[i]
                iris_x_train, iris_y_train = iris_x_train_skf[i], iris_y_train_skf[i]
                physical_x_train, physical_y_train = physical_x_train_skf[i], \
                        physical_y_train_skf[i]
                mfcc_ft_x_train, mfcc_ft_y_train = mfcc_ft_x_train_skf[i], mfcc_ft_y_train_skf[i]
                wav2vec_x_train, wav2vec_y_train = wav2vec_x_train_skf[i], wav2vec_y_train_skf[i]
                pert_x_train, pert_y_train = pert_x_train_skf[i], pert_y_train_skf[i]
                roberta_x_train, roberta_y_train = roberta_x_train_skf[i], roberta_y_train_skf[i]
                att_x_train, att_y_train = att_x_train_skf[i], att_y_train_skf[i]
                graph_x_train, graph_y_train = graph_x_train_skf[i], graph_y_train_skf[i]

                face_x_train, face_x_test, face_y_train, face_y_test = np.array(face_x_train), np.array(face_x_test_skf[i]),\
                        np.array(face_y_train), np.array(face_y_test_skf[i])
                face_vari_x_train, face_vari_x_test, face_vari_y_train, face_vari_y_test = np.array(face_vari_x_train), np.array(face_vari_x_test_skf[i]),\
                        np.array(face_vari_y_train), np.array(face_vari_y_test_skf[i])
                iris_x_train, iris_x_test, iris_y_train, iris_y_test = np.array(iris_x_train), np.array(iris_x_test_skf[i]),\
                        np.array(iris_y_train), np.array(iris_y_test_skf[i])
                physical_index_x_train, physical_index_x_test, physical_index_y_train, physical_index_y_test = \
                        np.array(physical_x_train), np.array(physical_x_test_skf[i]), np.array(physical_y_train), np.array(physical_y_test_skf[i])
                wav2vec_x_train, wav2vec_x_test, wav2vec_y_train, wav2vec_y_test = np.array(wav2vec_x_train), \
                        np.array(wav2vec_x_test_skf[i]), np.array(wav2vec_y_train), np.array(wav2vec_y_test_skf[i])
                mfcc_ft_x_train, mfcc_ft_x_test, mfcc_ft_y_train, mfcc_ft_y_test = np.array(mfcc_ft_x_train), \
                        np.array(mfcc_ft_x_test_skf[i]), np.array(mfcc_ft_y_train), np.array(mfcc_ft_y_test_skf[i])
                pert_x_train, pert_x_test, pert_y_train, pert_y_test = np.array(pert_x_train), np.array(pert_x_test_skf[i]),\
                        np.array(pert_y_train), np.array(pert_y_test_skf[i])
                roberta_x_train, roberta_x_test, roberta_y_train, roberta_y_test = np.array(roberta_x_train), np.array(roberta_x_test_skf[i]),\
                        np.array(roberta_y_train), np.array(roberta_y_test_skf[i])
                att_x_train, att_x_test, att_y_train, att_y_test = np.array(att_x_train), np.array(att_x_test_skf[i]),\
                        np.array(att_y_train), np.array(att_y_test_skf[i])
                graph_x_train, graph_x_test, graph_y_train, graph_y_test = np.array(graph_x_train), np.array(graph_x_test_skf[i]),\
                        np.array(graph_y_train), np.array(graph_y_test_skf[i])

                pert_x_train, pert_y_train = torch.from_numpy(pert_x_train).type(dtype), \
                        torch.from_numpy(pert_y_train).type(ltype)
                roberta_x_train, roberta_y_train = torch.from_numpy(roberta_x_train).type(dtype), \
                        torch.from_numpy(roberta_y_train).type(ltype)
                wav2vec_x_train, wav2vec_y_train = torch.from_numpy(wav2vec_x_train).type(dtype), \
                        torch.from_numpy(wav2vec_y_train).type(ltype)
                mfcc_ft_x_train, mfcc_ft_y_train = torch.from_numpy(mfcc_ft_x_train).type(dtype), \
                        torch.from_numpy(mfcc_ft_y_train).type(ltype)
                face_x_train, face_y_train = torch.from_numpy(face_x_train).type(dtype), \
                        torch.from_numpy(face_y_train).type(ltype)
                face_vari_x_train, face_vari_y_train = torch.from_numpy(face_vari_x_train).type(dtype), \
                        torch.from_numpy(face_vari_y_train).type(ltype)
                iris_x_train, iris_y_train = torch.from_numpy(iris_x_train).type(dtype), \
                        torch.from_numpy(iris_y_train).type(ltype)
                physical_index_x_train, physical_index_y_train = torch.from_numpy(physical_index_x_train).type(dtype), \
                        torch.from_numpy(physical_index_y_train).type(ltype)
                att_x_train, att_y_train = torch.from_numpy(att_x_train).type(dtype), \
                        torch.from_numpy(att_y_train).type(ltype)
                graph_x_train, graph_y_train = torch.from_numpy(graph_x_train).type(dtype), \
                        torch.from_numpy(graph_y_train).type(ltype)

                pert_x_test, pert_y_test = torch.from_numpy(pert_x_test).type(dtype), \
                        torch.from_numpy(pert_y_test).type(ltype)
                roberta_x_test, roberta_y_test = torch.from_numpy(roberta_x_test).type(dtype), \
                        torch.from_numpy(roberta_y_test).type(ltype)
                wav2vec_x_test, wav2vec_y_test = torch.from_numpy(wav2vec_x_test).type(dtype), \
                        torch.from_numpy(wav2vec_y_test).type(ltype)
                mfcc_ft_x_test, mfcc_ft_y_test = torch.from_numpy(mfcc_ft_x_test).type(dtype), \
                        torch.from_numpy(mfcc_ft_y_test).type(ltype)
                face_x_test, face_y_test = torch.from_numpy(face_x_test).type(dtype), \
                        torch.from_numpy(face_y_test).type(ltype)
                face_vari_x_test, face_vari_y_test = torch.from_numpy(face_vari_x_test).type(dtype), \
                        torch.from_numpy(face_vari_y_test).type(ltype)
                iris_x_test, iris_y_test = torch.from_numpy(iris_x_test).type(dtype), \
                        torch.from_numpy(iris_y_test).type(ltype)
                physical_index_x_test, physical_index_y_test = torch.from_numpy(physical_index_x_test).type(dtype), \
                        torch.from_numpy(physical_index_y_test).type(ltype)
                att_x_test, att_y_test = torch.from_numpy(att_x_test).type(dtype), \
                        torch.from_numpy(att_y_test).type(ltype)
                graph_x_test, graph_y_test = torch.from_numpy(graph_x_test).type(dtype), \
                        torch.from_numpy(graph_y_test).type(ltype)
                y_train = pert_y_train
                y_test = pert_y_test

                temp_acc = 0  
                device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
                model = EmbraceTextAudioVideo(device, [1404, 1404, 2776, 12987, 1024, 4722, 1024, 1024, 26365, 64], 32)
                files = os.listdir('./Result/overall/')
                for detail in files:
                        # print('./checkpoint/final/' + detail)
                        pt_name = 'Round_' + str(i)
                        if pt_name in detail and 'pth' in detail:
                                pt_path = './Result/overall/' + detail
                                best_acc = detail[-8:-4] 
                                temp_acc = float('0.' + best_acc)
                                print(temp_acc)
                                m_state_dict  = torch.load(pt_path, map_location=device)
                                model.load_state_dict(m_state_dict)
                
                model.to(device)
                optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
                # scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=4000, gamma=0.1)
                loss_func = torch.nn.CrossEntropyLoss()
                epochs = 100000
                for t in range(epochs+1):
                        model.train(True)
                        out, _ = model(roberta_x_train, pert_x_train, mfcc_ft_x_train, wav2vec_x_train, iris_x_train, \
                                physical_index_x_train, face_x_train, face_vari_x_train, att_x_train, graph_x_train)
                        loss = loss_func(out, y_train)
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
       
                        if t % 10 == 0 and t > 0:
     
                                prediction = []
                                for item in out:
                                        # print(item)
                                        if item[0] > item[1]:
                                                prediction.append(0)
                                        else:
                                                prediction.append(1)
                                pred_y = np.array(prediction)
                                target_y = y_train.cpu().data.numpy()
                                # print("train loss:%f" %loss.detach().cpu().numpy())
                                print("epoch %d, train acc %.4f" % (t, accuracy_score(pred_y, target_y)))

                                out, modality_indices = model(roberta_x_test, pert_x_test ,mfcc_ft_x_test, wav2vec_x_test, iris_x_test, 
                                        physical_index_x_test, face_x_test, face_vari_x_test, att_x_test, graph_x_test)
                                # print(type(modality_indices))
                                # print(modality_indices)
                                prediction = []
                                for item in out:
                                        # print(item)
                                        if item[0] > item[1]:
                                                prediction.append(0)
                                        else:
                                                prediction.append(1)
                                pred_y = np.array(prediction)
                                loss = loss_func(out,  y_test)
                                print("test loss:%f" %loss.detach().cpu().numpy())
                                target_y = y_test.cpu().data.numpy()
                                test_score = accuracy_score(target_y, pred_y)
                                test_f1 = f1_score(target_y, pred_y, average='weighted')
                                if test_score > temp_acc:
                                        acc =  test_score
                                        f1 = test_f1
                                        temp_acc = test_score
                                        torch.save(model.state_dict(), './Result/overall/Round_' + str(i) + '_' + str(int(round(acc, 4) * 10000)) + '.pth')
                                        torch.save(modality_indices, './Result/overall/Round_' + str(i) + '_indices.pt')
                                        # print(modality_indices)
                                        print('Round:{round} saved a model checkpoint at step: {step}. The best acc: {acc} and f1 score:{f1}'.format(round=i, time = j, step=t, acc=acc,f1=f1))
                                # print("test acc %.6f" % test_score)
                                
                #     print("test f1-score %f" %test_f1 )
                ACC.append(acc)
                F1.append(f1)
        print("LMF Max score:", np.max(ACC))
        print("LMF Min score:", np.min(ACC))
        print("LMF Avg score:", np.mean(ACC))
        print("LMF Max score:", np.max(F1))
        print("LMF Min score:", np.min(F1))
        print("LMF Avg F1-score:", np.mean(F1))
        

The 0 group of 0
0.7732
0.8247
0.8454
epoch 10, train acc 0.8243
test loss:0.664800
epoch 20, train acc 0.8278
test loss:0.656268
epoch 30, train acc 0.8370
test loss:0.657621
epoch 40, train acc 0.8381
test loss:0.645236
epoch 50, train acc 0.8312
test loss:0.646190
epoch 60, train acc 0.8347
test loss:0.637363
epoch 70, train acc 0.8473
test loss:0.635738
epoch 80, train acc 0.8404
test loss:0.635260
epoch 90, train acc 0.8450
test loss:0.632230
epoch 100, train acc 0.8393
test loss:0.626956
epoch 110, train acc 0.8393
test loss:0.636701
epoch 120, train acc 0.8393
test loss:0.639612
epoch 130, train acc 0.8450
test loss:0.629428
epoch 140, train acc 0.8335
test loss:0.627804
epoch 150, train acc 0.8450
test loss:0.627678
epoch 160, train acc 0.8439
test loss:0.618366
epoch 170, train acc 0.8462
test loss:0.613814
epoch 180, train acc 0.8393
test loss:0.625878
epoch 190, train acc 0.8404
test loss:0.603636
epoch 200, train acc 0.8416
test loss:0.620556
epoch 210, train acc 0.8370
tes

In [382]:
# Test
class EmbraceNet(nn.Module):
  
  def __init__(self, device, modality_indices, input_size_list, embracement_size=256, bypass_docking=False):
    """
    Initialize an EmbraceNet module.
    Args:
      device: A "torch.device()" object to allocate internal parameters of the EmbraceNet module.
      input_size_list: A list of input sizes.
      embracement_size: The length of the output of the embracement layer ("c" in the paper).
      bypass_docking: Bypass docking step, i.e., connect the input data directly to the embracement layer. If True, input_data must have a shape of [batch_size, embracement_size].
    """
    super(EmbraceNet, self).__init__()

    self.device = device
    self.input_size_list = input_size_list
    self.embracement_size = embracement_size
    self.bypass_docking = bypass_docking
    self.modality_indices = modality_indices
  
    if (not bypass_docking):
      for i, input_size in enumerate(input_size_list):
        # setattr(self, 'project_%d' % (i), nn.Linear(input_size, embracement_size ))
        setattr(self, 'docking_%d' % (i), nn.Linear(input_size , embracement_size))

  def forward(self, input_list, availabilities=None, selection_probabilities=None):
    """
    Forward input data to the EmbraceNet module.
    Args:
      input_list: A list of input data. Each input data should have a size as in input_size_list.
      availabilities: A 2-D tensor of shape [batch_size, num_modalities], which represents the availability of data for each modality. If None, it assumes that data of all modalities are available.
      selection_probabilities: A 2-D tensor of shape [batch_size, num_modalities], which represents probabilities that output of each docking layer will be selected ("p" in the paper). If None, the same probability of being selected will be used for each docking layer.
    Returns:
      A 2-D tensor of shape [batch_size, embracement_size] that is the embraced output.
    """

    # check input data
    assert len(input_list) == len(self.input_size_list)
    num_modalities = len(input_list)
    batch_size = input_list[0].shape[0]
    
    # docking layer
    docking_output_list = []
    if (self.bypass_docking):
      docking_output_list = input_list
    else:
      for i, input_data in enumerate(input_list):
        # x = getattr(self, 'project_%d' % (i))(input_data)
        x = getattr(self, 'docking_%d' % (i))(input_data)
         
        x = nn.functional.relu(x)
        docking_output_list.append(x)

    # check availabilities
    if (availabilities is None):
      availabilities = torch.ones(batch_size, len(input_list), dtype=torch.float, device=self.device)
    else:
      availabilities = availabilities.float()
    
    # adjust selection probabilities
    if (selection_probabilities is None):
      selection_probabilities = torch.ones(batch_size, len(input_list), dtype=torch.float, device=self.device)
    selection_probabilities = torch.mul(selection_probabilities, availabilities)
    
    probability_sum = torch.sum(selection_probabilities, dim=-1, keepdim=True)
    selection_probabilities = torch.div(selection_probabilities, probability_sum)
    # stack docking outputs
    docking_output_stack = torch.stack(docking_output_list, dim=-1)  # [batch_size, embracement_size, num_modalities]
    
    # print("Batch size:",batch_size)
    # embrace
#     modality_indices = torch.multinomial(selection_probabilities, num_samples=self.embracement_size, replacement=True)  # [batch_size, embracement_size]
#     print( self.modality_indices)
    modality_toggles = nn.functional.one_hot( self.modality_indices, num_classes=num_modalities).float()  # [batch_size, embracement_size, num_modalities]

    embracement_output_stack = torch.mul(docking_output_stack, modality_toggles)
    embracement_output = torch.sum(embracement_output_stack, dim=-1)  # [batch_size, embracement_size]
    return embracement_output


class EmbraceTextAudioVideo(nn.Module):
    def __init__(self, device, modality_indices, input_size, embrace_size):
        super(EmbraceTextAudioVideo, self).__init__()

        self.device = device
        # embracenet
        self.embracenet = EmbraceNet(device=self.device, modality_indices = modality_indices, input_size_list=input_size, embracement_size=embrace_size)

        # post embracement layers
        self.post = nn.Linear(in_features=embrace_size, out_features=2)

        # self.acti = nn.Tanhshrink()
        # self.acti = nn.Hardtanh()
        # self.acti = nn.ELU()
        # self.acti = nn.Softsign()
        # self.acti = nn.LogSigmoid()
        # self.acti = nn.Tanh()

    def forward(self, roberta, pert, wav, wav2vec, iris, physical_index, face, face_vari, att, graph):
        embraced_output = self.embracenet(input_list=[face, face_vari, iris, wav, wav2vec, physical_index, roberta, pert, att, graph])

        # embraced_output = self.embracenet(input_list=[roberta, pert, macbert, wav, iris, physical_index])
        x = self.post(embraced_output)
        # print(x)
        x = torch.sigmoid(x)
        # x = torch.relu(x)
        # x = torch.rrelu(x)
        # x = torch.selu(x)
        # x = torch.celu(x)
        # x = self.acti(x)
        return x


total_acc = []
total_f1 = []

if torch.cuda.is_available():
        dtype = torch.cuda.FloatTensor
        ltype = torch.cuda.LongTensor
else:
        dtype = torch.FloatTensor
        ltype = torch.LongTensor

for j in range(0, 1):
        ACC = []
        F1 = []
        for i in range(0, 10):
                print("The {round} group of {time}".format(round=i, time = j))

                face_x_train, face_y_train = face_x_train_skf[i], face_y_train_skf[i]
                face_vari_x_train, face_vari_y_train = face_vari_x_train_skf[i], face_vari_y_train_skf[i]
                iris_x_train, iris_y_train = iris_x_train_skf[i], iris_y_train_skf[i]
                physical_x_train, physical_y_train = physical_x_train_skf[i], \
                        physical_y_train_skf[i]
                mfcc_ft_x_train, mfcc_ft_y_train = mfcc_ft_x_train_skf[i], mfcc_ft_y_train_skf[i]
                wav2vec_x_train, wav2vec_y_train = wav2vec_x_train_skf[i], wav2vec_y_train_skf[i]
                pert_x_train, pert_y_train = pert_x_train_skf[i], pert_y_train_skf[i]
                roberta_x_train, roberta_y_train = roberta_x_train_skf[i], roberta_y_train_skf[i]
                att_x_train, att_y_train = att_x_train_skf[i], att_y_train_skf[i]
                graph_x_train, graph_y_train = graph_x_train_skf[i], graph_y_train_skf[i]

                face_x_train, face_x_test, face_y_train, face_y_test = np.array(face_x_train), np.array(face_x_test_skf[i]),\
                        np.array(face_y_train), np.array(face_y_test_skf[i])
                face_vari_x_train, face_vari_x_test, face_vari_y_train, face_vari_y_test = np.array(face_vari_x_train), np.array(face_vari_x_test_skf[i]),\
                        np.array(face_vari_y_train), np.array(face_vari_y_test_skf[i])
                iris_x_train, iris_x_test, iris_y_train, iris_y_test = np.array(iris_x_train), np.array(iris_x_test_skf[i]),\
                        np.array(iris_y_train), np.array(iris_y_test_skf[i])
                physical_index_x_train, physical_index_x_test, physical_index_y_train, physical_index_y_test = \
                        np.array(physical_x_train), np.array(physical_x_test_skf[i]), np.array(physical_y_train), np.array(physical_y_test_skf[i])
                wav2vec_x_train, wav2vec_x_test, wav2vec_y_train, wav2vec_y_test = np.array(wav2vec_x_train), \
                        np.array(wav2vec_x_test_skf[i]), np.array(wav2vec_y_train), np.array(wav2vec_y_test_skf[i])
                mfcc_ft_x_train, mfcc_ft_x_test, mfcc_ft_y_train, mfcc_ft_y_test = np.array(mfcc_ft_x_train), \
                        np.array(mfcc_ft_x_test_skf[i]), np.array(mfcc_ft_y_train), np.array(mfcc_ft_y_test_skf[i])
                pert_x_train, pert_x_test, pert_y_train, pert_y_test = np.array(pert_x_train), np.array(pert_x_test_skf[i]),\
                        np.array(pert_y_train), np.array(pert_y_test_skf[i])
                roberta_x_train, roberta_x_test, roberta_y_train, roberta_y_test = np.array(roberta_x_train), np.array(roberta_x_test_skf[i]),\
                        np.array(roberta_y_train), np.array(roberta_y_test_skf[i])
                att_x_train, att_x_test, att_y_train, att_y_test = np.array(att_x_train), np.array(att_x_test_skf[i]),\
                        np.array(att_y_train), np.array(att_y_test_skf[i])
                graph_x_train, graph_x_test, graph_y_train, graph_y_test = np.array(graph_x_train), np.array(graph_x_test_skf[i]),\
                        np.array(graph_y_train), np.array(graph_y_test_skf[i])

                pert_x_train, pert_y_train = torch.from_numpy(pert_x_train).type(dtype), \
                        torch.from_numpy(pert_y_train).type(ltype)
                roberta_x_train, roberta_y_train = torch.from_numpy(roberta_x_train).type(dtype), \
                        torch.from_numpy(roberta_y_train).type(ltype)
                wav2vec_x_train, wav2vec_y_train = torch.from_numpy(wav2vec_x_train).type(dtype), \
                        torch.from_numpy(wav2vec_y_train).type(ltype)
                mfcc_ft_x_train, mfcc_ft_y_train = torch.from_numpy(mfcc_ft_x_train).type(dtype), \
                        torch.from_numpy(mfcc_ft_y_train).type(ltype)
                face_x_train, face_y_train = torch.from_numpy(face_x_train).type(dtype), \
                        torch.from_numpy(face_y_train).type(ltype)
                face_vari_x_train, face_vari_y_train = torch.from_numpy(face_vari_x_train).type(dtype), \
                        torch.from_numpy(face_vari_y_train).type(ltype)
                iris_x_train, iris_y_train = torch.from_numpy(iris_x_train).type(dtype), \
                        torch.from_numpy(iris_y_train).type(ltype)
                physical_index_x_train, physical_index_y_train = torch.from_numpy(physical_index_x_train).type(dtype), \
                        torch.from_numpy(physical_index_y_train).type(ltype)
                att_x_train, att_y_train = torch.from_numpy(att_x_train).type(dtype), \
                        torch.from_numpy(att_y_train).type(ltype)
                graph_x_train, graph_y_train = torch.from_numpy(graph_x_train).type(dtype), \
                        torch.from_numpy(graph_y_train).type(ltype)

                pert_x_test, pert_y_test = torch.from_numpy(pert_x_test).type(dtype), \
                        torch.from_numpy(pert_y_test).type(ltype)
                roberta_x_test, roberta_y_test = torch.from_numpy(roberta_x_test).type(dtype), \
                        torch.from_numpy(roberta_y_test).type(ltype)
                wav2vec_x_test, wav2vec_y_test = torch.from_numpy(wav2vec_x_test).type(dtype), \
                        torch.from_numpy(wav2vec_y_test).type(ltype)
                mfcc_ft_x_test, mfcc_ft_y_test = torch.from_numpy(mfcc_ft_x_test).type(dtype), \
                        torch.from_numpy(mfcc_ft_y_test).type(ltype)
                face_x_test, face_y_test = torch.from_numpy(face_x_test).type(dtype), \
                        torch.from_numpy(face_y_test).type(ltype)
                face_vari_x_test, face_vari_y_test = torch.from_numpy(face_vari_x_test).type(dtype), \
                        torch.from_numpy(face_vari_y_test).type(ltype)
                iris_x_test, iris_y_test = torch.from_numpy(iris_x_test).type(dtype), \
                        torch.from_numpy(iris_y_test).type(ltype)
                physical_index_x_test, physical_index_y_test = torch.from_numpy(physical_index_x_test).type(dtype), \
                        torch.from_numpy(physical_index_y_test).type(ltype)
                att_x_test, att_y_test = torch.from_numpy(att_x_test).type(dtype), \
                        torch.from_numpy(att_y_test).type(ltype)
                graph_x_test, graph_y_test = torch.from_numpy(graph_x_test).type(dtype), \
                        torch.from_numpy(graph_y_test).type(ltype)
                y_train = pert_y_train
                y_test = pert_y_test

                device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
                files = os.listdir('./Result/overall/')
                for detail in files:
                        # print('./checkpoint/final/' + detail)
                        pt_name = 'Round_' + str(i)
                        if pt_name in detail and 'indices' in detail:
                                pt_path = './Result/overall/' + detail
                                print(pt_path)
                                load_indices = torch.load(pt_path)
                                model = EmbraceTextAudioVideo(device, load_indices, [1404, 1404, 2776, 12987, 1024, 4722, 1024, 1024, 26365, 64], 32)

                for detail in files:
                        # print('./checkpoint/final/' + detail)
                        pt_name = 'Round_' + str(i)
                        if pt_name in detail and '.pth' in detail:
                                pt_path = './Result/overall/' + detail
                                print(pt_path)
                                m_state_dict  = torch.load(pt_path, map_location=device)
                                model.load_state_dict(m_state_dict)
                                # print("//////////////////////////////////////////////////////")
                                # for parameters in model.parameters():
                                #         print(parameters)
                                # print("Model's state_dict:")
                                # for param_tensor in model.state_dict():
                                #         print(param_tensor, "\t", model.state_dict()[param_tensor].size())
                                # model.train(False
                model.to(device)
                out = model(roberta_x_test, pert_x_test ,mfcc_ft_x_test, wav2vec_x_test, iris_x_test, 
                                        physical_index_x_test, face_x_test, face_vari_x_test, att_x_test, graph_x_test)
                # out = model.forward(roberta_x_test, pert_x_test ,mfcc_ft_x_test, wav2vec_x_test, \
                #                         iris_x_test, physical_index_x_test, face_x_test, face_vari_x_test, graph_x_test)
                model.eval()
                # print(roberta_x_test[0])
                # print(out)      
                # print(out)
                prediction = []
                for item in out:
                        if item[0] > item[1]:
                                prediction.append(0)
                        else:
                                prediction.append(1)
                pred_y = np.array(prediction)
                
                target_y = y_test.cpu().data.numpy()
                test_score = accuracy_score(target_y, pred_y)
                test_f1 = f1_score(target_y, pred_y, average='weighted')
                print('The acc: {acc} and f1 score:{f1}'.format(acc=test_score,f1=test_f1))
        
                ACC.append(test_score)
                F1.append(test_f1)
        print("LMF Max score:", np.max(ACC))
        print("LMF Min score:z", np.min(ACC))
        print("LMF Avg score:", np.mean(ACC))
        print("LMF Max score:", np.max(F1))
        print("LMF Min score:", np.min(F1))
        print("LMF Avg F1-score:", np.mean(F1))


The 0 group of 0
./Result/overall/Round_0_indices.pt
./Result/overall/Round_0_7732.pth
./Result/overall/Round_0_8247.pth
./Result/overall/Round_0_8454.pth
The acc: 0.845360824742268 and f1 score:0.8097763627231479
The 1 group of 0
The acc: 0.8144329896907216 and f1 score:0.765161749022396
The 2 group of 0
The acc: 0.8144329896907216 and f1 score:0.765161749022396
The 3 group of 0
The acc: 0.8556701030927835 and f1 score:0.8270634903336893
The 4 group of 0
The acc: 0.7938144329896907 and f1 score:0.7299443061974168
The 5 group of 0
The acc: 0.8350515463917526 and f1 score:0.8055387103294926
The 6 group of 0
The acc: 0.8247422680412371 and f1 score:0.7770806552762337
The 7 group of 0
The acc: 0.845360824742268 and f1 score:0.8216433843713781
The 8 group of 0


RuntimeError: The size of tensor a (96) must match the size of tensor b (97) at non-singleton dimension 0